# Enhanced cohort analysis development

In this notebook we work on developing a collection of python methods relevant for the **enhanced cohort analysis**.
These methods will then be used in production phase to automatically build basic tree rules for all the available campaigns

1. [Loading modules](#Loading-modules)
2. [Temporary tables creation](#Temporary-tables-creation)
    * [Campaigns info extraction](#Campaigns-info-extraction)
    * [New campaigns identification](#New-campaigns-identification)
    * [Single campaign table creation](#Single-campaign-table-creation)
    * [Cinema Christmas Example](#Cinema-Christmas-Example)
3. [Data preprocessing](#Data-preprocessing)
    * [Preprocessing example](#Preprocessing-example)
4. [Rules creation utils](#Rules-creation-utils)
5. [Tree rules generation](#Tree-rules-generation)
    * [Rules generation example](#Rules-generation-example)
6. [Final formatting](#Final-formatting)
    * [Example of output table](#Example-of-output-table)
7. [Orchestration](#Orchestration)
8. [Single campaign example](#Single-campaign-example)

## Loading modules

In [106]:
import numpy as np
import pandas as pd

import logging
import os
import gcsfs
import time
import datetime
import dateutil.relativedelta
from google.cloud import bigquery

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

from statsmodels.stats.proportion import proportions_chisquare

from sklearn.tree import export_graphviz
from subprocess import call
from IPython.display import Image

## Global Vars

In [107]:
PROJECT_ID_1 = 'skyuk-uk-decis-models-01-dev'
DATASET_ID_1 = 'test_and_learn'
BUCKET_ID = 'fci02/test_and_learn'
CAMPAING_TABLE_NAME = 'optimus_base_clean'

## Temporary tables creation

We need to pull data from a table containing campaign informations, join with propensity mart table and then save it to GCS, so that we can load into memory and process data quickly.

### Campaigns info extraction

In [108]:
def create_channel_group(journey_row):
    
    tag = journey_row.lower()
    tag = ' '.join(tag.split('_'))
    
    if 'optin' in tag:
        tag = ' '.join(tag.split('optin'))
    if 'dmo' in tag:
        tag = ' '.join(tag.split('dmo')[1:])
 
    if ('em ' in tag) & ('dm' in tag) & ('obtm' in tag):
        return 'EM&DM&OBTM'
    elif ('em' in tag) & ('dm' in tag):
        return 'EM&DM'
    elif ('em' in tag) & ('obtm' in tag):
        return 'EM&OBTM'
    elif ('dm' in tag) & ('obtm' in tag):
        return 'DM&OBTM'
    elif ('em' in tag):
        return 'EM Only'
    elif ('dm' in tag):
        return 'DM Only'
    elif ('obtm' in tag):
        return 'OBTM Only'
    else:
        return 'UNKNOWN'

In [109]:
def format_campaigns_list(campaigns_frame):
    campaigns_frame = campaigns_frame.groupby(by=['campaign_program','campaign_name','start_date','journey','step','variant','max_step_end_reporting_date'], as_index=False).agg(lambda row:list(row))
    multiproduct_idx = campaigns_frame[campaigns_frame['product'].map(len)>1].index
    campaigns_frame.loc[multiproduct_idx,'product']=campaigns_frame.loc[multiproduct_idx,].apply(lambda row: ['multi'],axis=1)
    campaigns_frame['product'] = campaigns_frame['product'].apply(lambda row:row[0])
    
    campaigns_frame['channel'] = campaigns_frame['journey'].apply(lambda row:create_channel_group(row))
        
    today_date = datetime.date.today()
    # We pick up to latest available step date for each campaign
    # We consider a step completed if it is at least 4 days older than today date.
    # We use this information to define whether or not a given campaign is still running or not
    tmp = campaigns_list.groupby(by=['campaign_program','campaign_name'])[['max_step_end_reporting_date']].agg(lambda row: max(row))
    tmp['min_delta'] = (tmp['max_step_end_reporting_date']-datetime.date.today()).apply(lambda row: row.days)
    tmp['status'] = 'RUNNING'
    tmp.loc[tmp['min_delta']<-4,'status']='COMPLETED'
    campaigns_frame = pd.merge(campaigns_frame,tmp[['status']],on=['campaign_program','campaign_name'],how='inner')
    
    campaigns_frame.sort_values(by=['start_date'], inplace=True)
    campaigns_frame.reset_index(inplace=True, drop=True)
    
    return campaigns_frame

In [110]:
def get_campaigns_relevant_info(project_id: str, 
                                dataset_id: str,
                                table_id: str,
                                program_label: str,
                                campaign_label: str,
                                start_date_label: str,
                                product_label: str,
                                journey_label:str,
                                step_label:str,
                                variant_label:str,
                                step_end_reporting_date_label):
    """
    Extract basci information about campaigns that have been performed in the past, 
    and store them into a dataframe
    
    Parameters
    ----------
    
    project_id: str
        Name of the project on GCP
    dataset_id: str
        Name of the dataset the input table belongs
    table_id: str
        Name of the input table
    program_label: str
        Label used to define campaign program field
    campaign_label: str
        Label used to define campaign field
    start_date_label: str
        Label used to define campaign starting date field
    product_label: str
        Label used to define campaign product field
        
        
    Returns
    -------
    
    campaigns_frame: pandas.DataFrame
        Dataframe containing information about name, start date and product type of a given set of campaigns
    """
    
    bq = bigquery.Client()
    query = """
            SELECT {} as campaign_program,
                   {} as campaign_name,
                   {} as start_date,
                   {} as product,
                   {} as journey,
                   {} as step,
                   {} as variant,
                   max({}) as max_step_end_reporting_date
            from `{}.{}.{}`
            where {} not like 'Unknown%'
            group by {},
                     {},
                     {},
                     {},
                     {},
                     {},
                     {}
            """.format(program_label, campaign_label, start_date_label, product_label,journey_label,step_label,variant_label,step_end_reporting_date_label,
                       project_id, dataset_id, table_id,
                       product_label, 
                       program_label, campaign_label, start_date_label, product_label,journey_label,step_label,variant_label)

    campaigns_frame = bq.query(query).to_dataframe()
    campaigns_frame = format_campaigns_list(campaigns_frame)
    
    return campaigns_frame

In [228]:
def skim_campaigns(campaigns_table, backup_table):
    campaigns_analyzed = pd.unique(backup_table['campaign_name'])
    old_and_new_campaigns = pd.unique(campaigns_table['campaign_name'])
    
    to_analyze = list(set(old_and_new_campaigns).difference(set(campaigns_analyzed)))
    
    # First identify campaigns 
    new_campaigns_mask = campaigns_table['campaign_name'].apply(lambda row: True if row in to_analyze else False)
    
    # Now identify already analyzed campaigns that are still running
    still_running = pd.unique(backup_table.loc[backup_table['status']=='RUNNING','campaign_name']).tolist()
    running_mask = campaigns_table['campaign_name'].apply(lambda row: True if row in still_running else False)
    
    # Finally check whether or not new steps have been added
    tmp_1 = campaigns_table.groupby(by=['campaign_program','campaign_name'], as_index=False)[['max_step_end_reporting_date']].agg(lambda row: max(row))
    tmp_2  = backup_table.groupby(by=['campaign_program','campaign_name'])[['max_step_end_reporting_date']].agg(lambda row: max(row))
    tmp_3 = pd.merge(tmp_1,tmp_2,on=['campaign_program','campaign_name'],how='inner')
    check_mask = pd.to_datetime(tmp_3['max_step_end_reporting_date_x'])!=pd.to_datetime(tmp_3['max_step_end_reporting_date_y'])
    updated = pd.unique(tmp_3.loc[check_mask,'campaign_name']).tolist()
    updated_campaigns_mask = campaigns_table['campaign_name'].apply(lambda row: True if row in updated else False)
                
    mask = np.logical_or(np.logical_or(new_campaigns_mask,running_mask),updated_campaigns_mask)
    return campaigns_table[mask], still_running+updated, to_analyze

In [305]:
backup_table = campaigns_list.copy()
#backup_table['max_step_end_reporting_date'][0] = datetime.date(2020, 1, 18)
backup_table.loc[backup_table[backup_table.campaign_name=='Cinema Easter'].index,'status']='RUNNING'
to_drop = backup_table[backup_table.campaign_name=='Summer Cinema'].index
backup_table.drop(to_drop,inplace=True)
#to_drop = backup_table[backup_table.campaign_name=='Mobile'].index
#backup_table.drop(to_drop,inplace=True)
backup_table.to_csv('gs://'+BUCKET_ID+'/bckup_analyzed_campaigns_test_1.csv',index=False)

In [224]:
backup_table

,campaign_program,campaign_name,start_date,journey,step,variant,max_step_end_reporting_date,product,channel,status
0,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,OPTIMUS_WINBACK_JANUARY_EM1_3127-4_(DM241975)_...,2020-01-18,TV Winback,EM&DM,COMPLETED
1,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,OPTIMUS_WINBACK_JANUARY_EM8_3127-8_(DM243684)_...,2019-02-08,TV Winback,EM&DM,COMPLETED
2,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,OPTIMUS_WINBACK_JANUARY_EM8_3127-8_(DM243684)_...,2019-01-18,TV Winback,EM&DM,COMPLETED
3,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,OPTIMUS_WINBACK_JANUARY_EM8_3127-8_(DM243684)_...,2019-02-08,TV Winback,EM&DM,COMPLETED
4,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,OPTIMUS_WINBACK_JANUARY_EM8_3127-8_(DM243684)_...,2019-01-18,TV Winback,EM&DM,COMPLETED
...,...,...,...,...,...,...,...,...,...,...
22041,skyOptimusGetMore,Mobile Switch and Watch,2019-08-08,CAM0002944_Acquisition_Sky Mobile_UK_1920_Q3_O...,DM_CAM0002944_Sky_Mobile_Switch_Watch_July_201...,CAM0002944_SKY_MOBILE_SWITCH_AND_SAVE_DM_2019_...,2019-08-30,Mobile,DM Only,COMPLETED
22042,skyOptimusGetMore,Sky Stores Dumbo Hobbs and Shaw,2019-08-08,CAM0002829_Dumbo & Hobbs and Shaw_EMoptin,CAM0002829_Dumbo & Hobbs and Shaw_EMoptin-3,CAM0002829_SS_DHS_EM1A_B_CAM0002829_Short-Term...,2019-08-15,Broadband,EM Only,COMPLETED
22043,skyOptimusGetMore,Mobile Switch and Watch,2019-08-08,CAM0002944_Acquisition_Sky Mobile_UK_1920_Q3_O...,DM_CAM0002944_Sky_Mobile_Switch_Watch_July_201...,CAM0002944_SKY_MOBILE_SWITCH_AND_SAVE_DM_2019_...,2019-08-30,Mobile,DM Only,COMPLETED
22044,skyOptimusGetMore,Sports Range ELP,2019-08-26,CAM0002894 Week 36 Sports Range ELP,CAM0002894 Week 36 Sports Range ELP DM OPT IN DM,CAM0002894_WK36_RANGE_ELP_N/A20190816,2019-09-09,Sports,UNKNOWN,COMPLETED


In [229]:
skimmed, to_update, new = skim_campaigns(campaigns_list, backup_table)

In [230]:
to_update

['Cinema Easter', 'Winback DTV UK']

In [231]:
new

['Mobile', 'Summer Cinema']

In [182]:
skimmed

,campaign_program,campaign_name,start_date,journey,step,variant,max_step_end_reporting_date,product,channel,status
0,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,OPTIMUS_WINBACK_JANUARY_EM1_3127-4_(DM241975)_...,2019-01-18,TV Winback,EM&DM,COMPLETED
1,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,OPTIMUS_WINBACK_JANUARY_EM8_3127-8_(DM243684)_...,2019-02-08,TV Winback,EM&DM,COMPLETED
2,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,OPTIMUS_WINBACK_JANUARY_EM8_3127-8_(DM243684)_...,2019-01-18,TV Winback,EM&DM,COMPLETED
3,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,OPTIMUS_WINBACK_JANUARY_EM8_3127-8_(DM243684)_...,2019-02-08,TV Winback,EM&DM,COMPLETED
4,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,OPTIMUS_WINBACK_JANUARY_EM8_3127-8_(DM243684)_...,2019-01-18,TV Winback,EM&DM,COMPLETED
...,...,...,...,...,...,...,...,...,...,...
21969,skyOptimusGetMore,Summer Cinema,2019-07-15,CAM0002842 Summer Cinema EM | EMDM Opt In,CAM0002842 SUMMER CINEMA EM OPT IN EMDM2 ALL UK,CAM0002884_SUMMER_CINEMA_TRADING-4681-5_(DM323...,2019-08-08,Cinema,EM&DM,COMPLETED
21970,skyOptimusGetMore,Summer Cinema,2019-07-15,CAM0002842 Summer Cinema EM | EMDM Opt In,CAM0002842 SUMMER CINEMA FOLLOW UP EM OPT IN E...,CAM0002884_SUMMER_CINEMA_TRADING_FOLLOW_UP-468...,2019-08-16,Cinema,EM&DM,COMPLETED
21971,skyOptimusGetMore,Summer Cinema,2019-07-15,CAM0002842 Summer Cinema EM | EMDM Opt In,CAM0002842 SUMMER CINEMA FOLLOW UP EM OPT IN E...,CAM0002884_SUMMER_CINEMA_TRADING_FOLLOW_UP-468...,2019-08-22,Cinema,EM&DM,COMPLETED
21972,skyOptimusGetMore,Summer Cinema,2019-07-15,CAM0002842 Summer Cinema EM | EMDM Opt In,CAM0002842 SUMMER CINEMA EM OPT IN EMDM2 ALL UK,CAM0002884_SUMMER_CINEMA_TRADING-4681-5_(DM323...,2019-08-08,Cinema,EM&DM,COMPLETED


In [297]:
sliced = campaigns_list[campaigns_list.campaign_name=='Winback DTV UK']
sliced

,campaign_program,campaign_name,start_date,journey,step,variant,max_step_end_reporting_date,product,channel,status
0,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,OPTIMUS_WINBACK_JANUARY_EM1_3127-4_(DM241975)_...,2019-01-18,TV Winback,EM&DM,COMPLETED
1,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,OPTIMUS_WINBACK_JANUARY_EM8_3127-8_(DM243684)_...,2019-02-08,TV Winback,EM&DM,COMPLETED
2,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,OPTIMUS_WINBACK_JANUARY_EM8_3127-8_(DM243684)_...,2019-01-18,TV Winback,EM&DM,COMPLETED
3,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,OPTIMUS_WINBACK_JANUARY_EM8_3127-8_(DM243684)_...,2019-02-08,TV Winback,EM&DM,COMPLETED
4,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,OPTIMUS_WINBACK_JANUARY_EM8_3127-8_(DM243684)_...,2019-01-18,TV Winback,EM&DM,COMPLETED
...,...,...,...,...,...,...,...,...,...,...
365,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,OPTIMUS_WINBACK_DECEMBER_EM1_2737-14_(DM231511...,2018-12-21,TV Winback,EM&DM,COMPLETED
367,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,DELIVERY_CAM0002279_EM_Ents_SkyQ_Reactive_DMEM...,2018-11-16,TV Winback,EM&DM,COMPLETED
368,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,DELIVERY_CAM0002279_EM_Ents_SkyPlus_Reactive_D...,2018-11-16,TV Winback,EM&DM,COMPLETED
369,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,03_CAM0002279_DMO_Winback_DMEM_OptIn_UK_2018_DM4,CAM0002279_REINSTATE_DTV_DM_4_UK_1819_N/A20190109,2019-06-16,TV Winback,EM&DM,COMPLETED


In [299]:
lobo=pd.concat([campaigns_list, sliced])
lobo.drop_duplicates()

,campaign_program,campaign_name,start_date,journey,step,variant,max_step_end_reporting_date,product,channel,status
0,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,OPTIMUS_WINBACK_JANUARY_EM1_3127-4_(DM241975)_...,2019-01-18,TV Winback,EM&DM,COMPLETED
1,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,OPTIMUS_WINBACK_JANUARY_EM8_3127-8_(DM243684)_...,2019-02-08,TV Winback,EM&DM,COMPLETED
2,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,OPTIMUS_WINBACK_JANUARY_EM8_3127-8_(DM243684)_...,2019-01-18,TV Winback,EM&DM,COMPLETED
3,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,OPTIMUS_WINBACK_JANUARY_EM8_3127-8_(DM243684)_...,2019-02-08,TV Winback,EM&DM,COMPLETED
4,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,OPTIMUS_WINBACK_JANUARY_EM8_3127-8_(DM243684)_...,2019-01-18,TV Winback,EM&DM,COMPLETED
...,...,...,...,...,...,...,...,...,...,...
22041,skyOptimusGetMore,Mobile Switch and Watch,2019-08-08,CAM0002944_Acquisition_Sky Mobile_UK_1920_Q3_O...,DM_CAM0002944_Sky_Mobile_Switch_Watch_July_201...,CAM0002944_SKY_MOBILE_SWITCH_AND_SAVE_DM_2019_...,2019-08-30,Mobile,DM Only,COMPLETED
22042,skyOptimusGetMore,Sky Stores Dumbo Hobbs and Shaw,2019-08-08,CAM0002829_Dumbo & Hobbs and Shaw_EMoptin,CAM0002829_Dumbo & Hobbs and Shaw_EMoptin-3,CAM0002829_SS_DHS_EM1A_B_CAM0002829_Short-Term...,2019-08-15,Broadband,EM Only,COMPLETED
22043,skyOptimusGetMore,Mobile Switch and Watch,2019-08-08,CAM0002944_Acquisition_Sky Mobile_UK_1920_Q3_O...,DM_CAM0002944_Sky_Mobile_Switch_Watch_July_201...,CAM0002944_SKY_MOBILE_SWITCH_AND_SAVE_DM_2019_...,2019-08-30,Mobile,DM Only,COMPLETED
22044,skyOptimusGetMore,Sports Range ELP,2019-08-26,CAM0002894 Week 36 Sports Range ELP,CAM0002894 Week 36 Sports Range ELP DM OPT IN DM,CAM0002894_WK36_RANGE_ELP_N/A20190816,2019-09-09,Sports,UNKNOWN,COMPLETED


In [318]:
campaigns_list['max_step_end_reporting_date'].values

array([datetime.date(2019, 1, 18), datetime.date(2019, 2, 8),
       datetime.date(2019, 1, 18), ..., datetime.date(2019, 8, 30),
       datetime.date(2019, 9, 9), datetime.date(2019, 9, 9)], dtype=object)

In [317]:
taba = pd.read_csv('gs://'+BUCKET_ID+'/bckup_analyzed_campaigns_test_1.csv')
#taba.drop_duplicates(subset=taba.columns.drop('max_step_end_reporting_date'),inplace=True)
taba['start_date'].values

array(['2018-12-14', '2018-12-14', '2018-12-14', ..., '2018-12-28',
       '2018-12-28', '2018-12-28'], dtype=object)

In [295]:
pd.unique(taba['max_step_end_reporting_date'])

array(['2018-12-28', '2018-12-29', '2019-01-05', '2019-01-11',
       '2018-11-23', '2018-11-30', '2018-12-07', '2018-12-09',
       '2018-12-16', '2018-12-27', '2019-01-17', '2019-01-18',
       '2019-01-24', '2019-01-31', '2019-02-01', '2019-02-07',
       '2019-02-08', '2019-02-12', '2019-02-14', '2019-02-15',
       '2019-02-21', '2019-02-22', '2019-03-01', '2019-03-08',
       '2019-03-14', '2019-03-15', '2019-03-21', '2019-03-28',
       '2019-03-30', '2019-04-04', '2019-04-11', '2019-04-12',
       '2019-04-17', '2019-04-18', '2019-04-19', '2019-04-25',
       '2019-04-26', '2019-05-03', '2019-05-09', '2019-05-10',
       '2019-05-16', '2019-05-17', '2019-05-23', '2019-05-24',
       '2019-05-30', '2019-05-31', '2019-06-06', '2019-06-07',
       '2019-06-13', '2019-06-14', '2019-06-20', '2019-06-21',
       '2019-06-27', '2019-06-29', '2019-07-04', '2019-07-11',
       '2019-07-12', '2019-07-18', '2019-07-19', '2019-07-25',
       '2019-07-26', '2019-08-01', '2019-08-02', '2019-

In [277]:
taba['max_step_end_reporting_date'].max()

'2020-01-18'

In [303]:
pd.unique(taba.status)

array(['COMPLETED', 'RUNNING'], dtype=object)

In [249]:
campaigns_list

,campaign_program,campaign_name,start_date,journey,step,variant,max_step_end_reporting_date,product,channel,status
0,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,OPTIMUS_WINBACK_JANUARY_EM1_3127-4_(DM241975)_...,2019-01-18,TV Winback,EM&DM,COMPLETED
1,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,OPTIMUS_WINBACK_JANUARY_EM8_3127-8_(DM243684)_...,2019-02-08,TV Winback,EM&DM,COMPLETED
2,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,OPTIMUS_WINBACK_JANUARY_EM8_3127-8_(DM243684)_...,2019-01-18,TV Winback,EM&DM,COMPLETED
3,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,OPTIMUS_WINBACK_JANUARY_EM8_3127-8_(DM243684)_...,2019-02-08,TV Winback,EM&DM,COMPLETED
4,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,OPTIMUS_WINBACK_JANUARY_EM8_3127-8_(DM243684)_...,2019-01-18,TV Winback,EM&DM,COMPLETED
...,...,...,...,...,...,...,...,...,...,...
22041,skyOptimusGetMore,Mobile Switch and Watch,2019-08-08,CAM0002944_Acquisition_Sky Mobile_UK_1920_Q3_O...,DM_CAM0002944_Sky_Mobile_Switch_Watch_July_201...,CAM0002944_SKY_MOBILE_SWITCH_AND_SAVE_DM_2019_...,2019-08-30,Mobile,DM Only,COMPLETED
22042,skyOptimusGetMore,Sky Stores Dumbo Hobbs and Shaw,2019-08-08,CAM0002829_Dumbo & Hobbs and Shaw_EMoptin,CAM0002829_Dumbo & Hobbs and Shaw_EMoptin-3,CAM0002829_SS_DHS_EM1A_B_CAM0002829_Short-Term...,2019-08-15,Broadband,EM Only,COMPLETED
22043,skyOptimusGetMore,Mobile Switch and Watch,2019-08-08,CAM0002944_Acquisition_Sky Mobile_UK_1920_Q3_O...,DM_CAM0002944_Sky_Mobile_Switch_Watch_July_201...,CAM0002944_SKY_MOBILE_SWITCH_AND_SAVE_DM_2019_...,2019-08-30,Mobile,DM Only,COMPLETED
22044,skyOptimusGetMore,Sports Range ELP,2019-08-26,CAM0002894 Week 36 Sports Range ELP,CAM0002894 Week 36 Sports Range ELP DM OPT IN DM,CAM0002894_WK36_RANGE_ELP_N/A20190816,2019-09-09,Sports,UNKNOWN,COMPLETED


In [218]:
pd.unique(backup_table['campaign_name']).shape

(59,)

### New campaigns identification

We run analysis only for those campaigns that have not been processed yet.

In [118]:
def get_skimmed_campaigns(campaigns_table: pd.DataFrame,
                          project_id: str,
                          bucket_id: str,
                          campaigns_bkup_table_name: str,
                          prefix='gs://'):
      
    """
    Look for a back up table containing the information regarding campaigns that have been already analyzed.
    If such table exists then use this information to reduce the number of campaigns to analyze
    
    Parameters
    ----------
    
    campaigns_table: pandas.DataFrame
        Dataframe containing information about name, start date and product type of a given set of campaigns
    project_id: str
        Name of the project on GCP
    bucket_id: str
        Name of the bucket on GCP
    campaigns_bkup_table_path_namr: str
        Name of the back up table
    
    
    Returns
    -------
    
    campaigns_table: pandas.DataFrame
        Dataframe containing information about name, start date and product type of campaigns that have not been analyzed before
    """
    
    fs=gcsfs.GCSFileSystem(project=project_id)
    table_path = os.path.join(bucket_id,campaigns_bkup_table_name)

    if fs.exists(prefix+table_path):
        backup_table = pd.read_csv(prefix+table_path)
        if len(campaigns_table.columns.difference(backup_table.columns)):
            raise ValueError('The campaigns list table and backup table must have the same type of columns')
        elif len(campaigns_table)<len(backup_table):
            raise ValueError('The campaigns list table has cannot be outdated w.r.t. the backup table')
        else:            
            return skim_campaigns(campaigns_table, backup_table)

    else:

        return campaigns_table 

### Single campaign table creation

For a given campaign we select the corresponding customers, and then retrieve customers product holding features from a time period just before the beginning of the campaign

In [10]:
def create_campaign_analysis_table(project_id_1: str,  dataset_id_1: str, table_id_1: str,
                                   optimus_columns: dict, program_name: str, campaign_name: str,  
                                   target_labels: str, list_of_filters: tuple, analysis_level,
                                   project_id_2: str,  dataset_id_2: str, table_id_2: str,
                                   prop_mart_features: list, base_date_label: str, base_date_value: str,
                                   selection_eq: str, control_group_name: str,
                                   out_table_name: str,
                                   step=None):
    
    list_of_features = ','.join(prop_mart_features)
    grouping_features = list_of_features + ",b.{}".format(optimus_columns['campaign_label'])
    
    list_of_final_features = grouping_features+', max(b.customer_prod_conversion) as target'
    list_of_filters=list('"{}"'.format(filtro) for filtro in list_of_filters)
    list_of_filters = ','.join(list_of_filters)
    
    target_label = ['coalesce(cast({} as INT64),0)'.format(label) for label in target_labels]
    target_label = '+'.join(target_label)
    
    campaign_query = """CREATE OR REPLACE TABLE `{}.{}.{}` AS (
                        WITH campaign_conversions as(
                        SELECT
                          {},
                          {},
                        CASE WHEN SUM({}) >=1 THEN 1 ELSE 0 END AS customer_prod_conversion
                        from   `{}.{}.{}`
                         WHERE
                           ((({} {} 'Treatment')
                                  AND ({}='{}'))
                                  AND ({} = '{}')
                                  AND({} = '{}')""".format(project_id_1,dataset_id_1,out_table_name,
                                                               optimus_columns['campaign_label'],optimus_columns['account_number_label'],target_label,
                                                               project_id_1,dataset_id_1,table_id_1,
                                                               optimus_columns['cell_label'],selection_eq,
                                                               optimus_columns['control_label'],control_group_name,
                                                               optimus_columns['program_label'], program_name,
                                                               optimus_columns['campaign_label'], campaign_name)
    if (analysis_level=='variant_label') & (selection_eq=='!=') & (step!=None):
        filter_query = """ 
                       AND ({} in ('{}')))
                       group by   {}, {}),""".format(optimus_columns['step_label'],step,
                                                     optimus_columns['campaign_label'], optimus_columns['account_number_label'])
    else:
          filter_query = """ 
                       AND ({} in ({})))
                       group by   {}, {}),""".format(optimus_columns[analysis_level],list_of_filters,
                                                     optimus_columns['campaign_label'], optimus_columns['account_number_label'])  
    prop_mart_query="""
            prop_mart as(
            select {} from `{}.{}.{}`
            where ({} = '{}'))""".format(list_of_features, project_id_2,  dataset_id_2, table_id_2,
                                         base_date_label, base_date_value)

    final_query="""
            select {}
            from prop_mart 
            inner join campaign_conversions b using({})
            group by {})
            """.format(list_of_final_features,
                       optimus_columns['account_number_label'],
                       grouping_features)
    
    query=campaign_query+filter_query+prop_mart_query+final_query
    #print(query)

    try:
        bq = bigquery.Client(project=project_id_1)
        dest_dataset_ref = bq.dataset(dataset_id_1)
        dest_table_ref = dest_dataset_ref.table(out_table_name)
        #job_config = bigquery.QueryJobConfig()
        #job_config.allow_large_results = True               # Allow large query results tables (legacy SQL, only)
        #job_config.destination = None             # Table where results are written or None if not set
        #job_config.create_disposition = 'CREATE_IF_NEEDED'  # Create the table if it does not exist
        #job_config.use_query_cache = False
        query_job = bq.query(query) #job_config=job_config)
        query_job.result()
        print('Created BigQuery table: {}.{}.{}'.format(project_id_1, dataset_id_1, out_table_name))
    except Exception as e:
        e = str(e)
        raise RuntimeError(e)
    
    #bq = bigquery.Client()
    #out = bq.query(query)
    #print(out)

In [11]:
def pull_table(campaign_data_paths,
               prop_mart_data_paths,
               subset_row,
               target_labels,
               analysis_level,
               optimus_columns,
               prop_mart_features,
               base_date_label,
               prop_mart_date,
               relation,
               control_group_name,
               table_name,
               bucket_id,
               step=None):

    create_campaign_analysis_table(campaign_data_paths['project_id'], campaign_data_paths['dataset_id'], campaign_data_paths['table_id'],
                                   optimus_columns,
                                   subset_row.loc['campaign_program'], subset_row.loc['campaign_name'],target_labels,subset_row.loc[analysis_level.split('_')[0]],analysis_level,
                                   prop_mart_data_paths['project_id'], prop_mart_data_paths['dataset_id'], prop_mart_data_paths['table_id'],
                                   prop_mart_features, base_date_label, prop_mart_date,
                                   relation, control_group_name, table_name,step)
    gcs_treat_path ='gs://'+os.path.join(bucket_id,table_name+'.csv')
    export_query_to_gcs(campaign_data_paths['project_id'], campaign_data_paths['dataset_id'],
                        table_name,gcs_treat_path)
    return pd.read_csv(gcs_treat_path)

### Single campaign table extraction to GCS

Once a campaing table has been created on BigQuery, we save it to GCS

In [12]:
def export_query_to_gcs(project_id, dataset_id, table_id, bucket, bucket_location='EU', file_format='CSV'):
    """
    Export the contents of a BigQuery table to CSV files inside a GCS bucket.
    Note: requires READER permissions to the source BigQuery dataset and WRITER permissions to the GCS bucket.
    :param string project_id:
    :param string dataset_id:
    :param string table_id:
    :param bucket: the destination URI on GCS (e.g. gs://{bucket_name}/{folder_name}/{file_name}_*.csv)
    :param bucket_location: the destination's bucket location (must match the source table's location)
    :param file_format: final file format. expected values = CSV, NEWLINE_DELIMITED_JSON 
    :return:
    """
    
    bq = bigquery.Client(project=project_id)
    dataset_ref = bq.dataset(dataset_id)
    table_ref = dataset_ref.table(table_id)
    job_config = bigquery.job.ExtractJobConfig()
    job_config.destination_format = file_format
    extract_job = bq.extract_table(
        table_ref,
        bucket,
        location=bucket_location,
        job_config=job_config
    )  # API request
    extract_job.result()
    print('Exported {}:{}.{} to {}'.format(project_id, dataset_id, table_id, bucket))

### Cinema Christmas Example

An example. of how the data extraction should work for a single campaign

In [44]:
OPTIMUS_COLS = {'program_label':'L1_programme',
                'campaign_label':'L2_Campaign',
                'journey_label':'L3_Journey',
                'step_label':'L4_Step',
                'variant_label':'L5_Variant',
                'product_label':'product',
                'account_number_label':'account_number',
                'sale_vol_label':'sale_vol',
                'sale_aux_vol_label':'sale_auxillary_vol',
                'cell_label':'cell_type',
                'control_label':'control_type2',
                'start_date_label':'CAMPAIGN_REPORTING_START_DATE',
                'end_date_label':'STEP_REPORTING_END_DATE',
                'channel_label':'CHANNEL'}

In [103]:
campaigns_list = get_campaigns_relevant_info(PROJECT_ID_1,DATASET_ID_1,CAMPAING_TABLE_NAME,
                                             OPTIMUS_COLS['program_label'],
                                             OPTIMUS_COLS['campaign_label'],
                                             OPTIMUS_COLS['start_date_label'], 
                                             OPTIMUS_COLS['product_label'],
                                             OPTIMUS_COLS['journey_label'],
                                             OPTIMUS_COLS['step_label'],
                                             OPTIMUS_COLS['variant_label'],
                                             OPTIMUS_COLS['end_date_label']
                                            )
campaigns_list.head(10)

,campaign_program,campaign_name,start_date,journey,step,variant,max_step_end_reporting_date,product,channel,status
0,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,OPTIMUS_WINBACK_JANUARY_EM1_3127-4_(DM241975)_...,2019-01-18,TV Winback,EM&DM,COMPLETED
1,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,OPTIMUS_WINBACK_JANUARY_EM8_3127-8_(DM243684)_...,2019-02-08,TV Winback,EM&DM,COMPLETED
2,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,OPTIMUS_WINBACK_JANUARY_EM8_3127-8_(DM243684)_...,2019-01-18,TV Winback,EM&DM,COMPLETED
3,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,OPTIMUS_WINBACK_JANUARY_EM8_3127-8_(DM243684)_...,2019-02-08,TV Winback,EM&DM,COMPLETED
4,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,OPTIMUS_WINBACK_JANUARY_EM8_3127-8_(DM243684)_...,2019-01-18,TV Winback,EM&DM,COMPLETED
5,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,OPTIMUS_WINBACK_JANUARY_EM8_3127-8_(DM243684)_...,2019-02-08,TV Winback,EM&DM,COMPLETED
6,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,OPTIMUS_WINBACK_JANUARY_EM8_3127-8_(DM243684)_...,2019-01-18,TV Winback,EM&DM,COMPLETED
7,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,OPTIMUS_WINBACK_DECEMBER_EM8-2737-18_(DM231961...,2018-12-21,TV Winback,EM&DM,COMPLETED
8,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,OPTIMUS_WINBACK_DECEMBER_EM8-2737-18_(DM231961...,2018-12-12,TV Winback,EM&DM,COMPLETED
9,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,OPTIMUS_WINBACK_DECEMBER_EM8-2737-18_(DM231961...,2018-12-21,TV Winback,EM&DM,COMPLETED


In [104]:
campaigns_list.shape

(22046, 10)

In [105]:
pd.unique(campaigns_list.status)

array(['COMPLETED'], dtype=object)

In [56]:
((campaigns_list['max_step_end_reporting_date']-datetime.date.today()).apply(lambda row: row.days)<-4).sum()

22046

In [14]:
pd.unique(campaigns_list[campaigns_list.campaign_name=='Winback DTV UK']['start_date'])

array([datetime.date(2018, 10, 15)], dtype=object)

In [221]:
campaigns_list.shape

(22046, 8)

In [263]:
taba=pd.read_csv('gs://'+BUCKET_ID+'/bckup_analyzed_campaigns_test_1.csv').sort_values(by=['campaign_program','campaign_name','start_date','max_step_end_reporting_date']).drop_duplicates(keep='last')
taba[taba.campaign_name=='Winback DTV UK']

,campaign_program,campaign_name,start_date,journey,step,variant,max_step_end_reporting_date,product,channel,status
210,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_EMOptin_Marketing_EM8,DELIVERY_EM8_SKY+SPORTS_CAM0002279_EM_Ents_Spo...,2018-10-22,TV Winback,EM Only,RUNNING
308,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,DELIVERY_EM1_DM_SKY_Q_NOSPORTS_CAM0002279_EM_E...,2018-10-26,TV Winback,EM&DM,RUNNING
309,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,DELIVERY_EM1_DM_SKY+_NO_SPORTS_CAM0002279_EM_E...,2018-10-26,TV Winback,EM&DM,RUNNING
311,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,DELIVERY_CAM0002279_EM_Ents_Sports_SkyQ_Reacti...,2018-10-26,TV Winback,EM&DM,RUNNING
243,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_EMOptin_Marketing_EM5,DELIVERY_EM5_SKY+_NO_SPORTS_CAM0002279_EM_Ents...,2018-11-10,TV Winback,EM Only,RUNNING
...,...,...,...,...,...,...,...,...,...,...
302,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,03_CAM0002279_DMO_Winback_DMEM_OptIn_UK_2018_DM4,CAM0002279_DTV_WINBACK_EMDM_4_UK_CAM0002279_DM...,2019-06-16,TV Winback,EM&DM,RUNNING
304,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,03_CAM0002279_DMO_Winback_DMEM_OptIn_UK_2018_DM2,CAM0002279_DTV_WINBACK_EMDM_2_UK_CAM0002279_DM...,2019-06-16,TV Winback,EM&DM,RUNNING
306,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,03_CAM0002279_DMO_Winback_DMEM_OptIn_UK_2018_DM4,CAM0002279_DTV_WINBACK_EMDM_4_UK_CAM0002279_DM...,2019-06-16,TV Winback,EM&DM,RUNNING
315,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,03_CAM0002279_DMO_Winback_DMEM_OptIn_UK_2018_DM4,CAM0002279_REINSTATE_DTV_DM_4_UK_1819_N/A20190109,2019-06-16,TV Winback,EM&DM,RUNNING


In [15]:
def create_right_granularity_level(campaigns_table,specific_campaign, granularity_level):

    temp_table = pd.merge(campaigns_table, specific_campaign, on=['campaign_program','campaign_name'],how='inner')

    main_keys = ['campaign_program','campaign_name','start_date','product']
    if granularity_level=='overall':
        temp_table = temp_table[main_keys+['journey']].drop_duplicates()
        temp_table = temp_table.groupby(by=main_keys, as_index=False).agg(lambda row:list(row))
        return temp_table,'journey_label'
    elif granularity_level=='channel':
        temp_table = temp_table[main_keys+['channel','journey']].drop_duplicates()
        temp_table = temp_table.groupby(by=main_keys+['channel'],as_index=False).agg(lambda row:list(row))
        return temp_table,'journey_label'
    elif granularity_level=='journey':
        temp_table = temp_table[main_keys+['channel','journey']].drop_duplicates().sort_values(by=main_keys+['channel','journey'])
        temp_table['journey']=temp_table['journey'].apply(lambda row:[row])
        return temp_table,'journey_label'
    elif granularity_level=='step':
        temp_table = temp_table[main_keys+['channel','journey','step']].drop_duplicates().sort_values(by=main_keys+['channel','journey','step'])
        temp_table['step']=temp_table['step'].apply(lambda row:[row])
        return temp_table,'step_label'
    elif granularity_level=='variant':
        temp_table = temp_table.sort_values(by=main_keys+['channel','journey','step','variant'])
        temp_table['variant']=temp_table['variant'].apply(lambda row:[row])
        return temp_table,'variant_label'
    else:
        raise ValueError('Invalid granularity value provided')

In [16]:
def provide_analysis_level_rules(campaigns_table,specific_campaign):
    
    temp_table = pd.merge(campaigns_table, specific_campaign, on=['campaign_program','campaign_name'],how='inner')
    main_keys = ['campaign_program','campaign_name','start_date','product']
    analysis_level_rules={'overall':True}
    
    to_add = []
    for analysis_level in ['channel','journey','step','variant']:
        to_add+=[analysis_level]
        extended_keys=main_keys+to_add
        reduced_table = temp_table[extended_keys].drop_duplicates()
       
        value = reduced_table.groupby(by=extended_keys[:-1])[analysis_level].count().max()
        print(value)
        if value==1:
            analysis_level_rules[analysis_level]=False
        else:
            analysis_level_rules[analysis_level]=True
    return analysis_level_rules

In [17]:
create_right_granularity_level(campaigns_list,campaigns_list[['campaign_program','campaign_name']][0:1], 'overall')[0]

,campaign_program,campaign_name,start_date,product,journey
0,skyOptimusDMO,Winback DTV UK,2018-10-15,TV Winback,"[CAM0002279 DMO Winback DM EM OptIn UK 2018, C..."


In [276]:
create_right_granularity_level(campaigns_list,campaigns_list[['campaign_program','campaign_name']][0:1], 'journey')[0]

,campaign_program,campaign_name,start_date,product,channel,journey
44,skyOptimusDMO,Winback DTV UK,2018-10-15,TV Winback,DM Only,[CAM0002279 DMO Winback DM OptIn UK 2018]
33,skyOptimusDMO,Winback DTV UK,2018-10-15,TV Winback,EM Only,[CAM0002279 DMO Winback EM OptIn UK 2018]
0,skyOptimusDMO,Winback DTV UK,2018-10-15,TV Winback,EM&DM,[CAM0002279 DMO Winback DM EM OptIn UK 2018]
59,skyOptimusDMO,Winback DTV UK,2018-10-15,TV Winback,EM&DM,[CAM0002279 DMO Winback DM EM OptIn UK March R...
37,skyOptimusDMO,Winback DTV UK,2018-10-15,TV Winback,OBTM Only,[CAM0002279 DMO Winback OBTM UK 2018]


In [262]:
create_right_granularity_level(campaigns_list,campaigns_list[['campaign_program','campaign_name']][0:1], 'variant')[0]

,campaign_program,campaign_name,start_date,journey,step,variant,product,channel
58,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM OptIn UK 2018,01_CAM0002279_DMO_Winback_DM_OptIn_UK_2018_DM_1,[CAM0002279_DTV_WINBACK_DM_1_UK_CAM0002279_DM_...,TV Winback,DM Only
57,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM OptIn UK 2018,01_CAM0002279_DMO_Winback_DM_OptIn_UK_2018_DM_1,[CAM0002279_DTV_WINBACK_DM_1_UK_CAM0002279_DM_...,TV Winback,DM Only
56,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM OptIn UK 2018,01_CAM0002279_DMO_Winback_DM_OptIn_UK_2018_DM_1,[CAM0002279_DTV_WINBACK_DM_1_UK_CAM0002279_DM_...,TV Winback,DM Only
55,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM OptIn UK 2018,01_CAM0002279_DMO_Winback_DM_OptIn_UK_2018_DM_1,[CAM0002279_DTV_WINBACK_DM_1_UK_CAM0002279_DM_...,TV Winback,DM Only
54,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM OptIn UK 2018,01_CAM0002279_DMO_Winback_DM_OptIn_UK_2018_DM_1,[CAM0002279_DTV_WINBACK_DM_1_UK_CAM0002279_DM_...,TV Winback,DM Only
...,...,...,...,...,...,...,...,...
39,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback OBTM UK 2018,CAM0002279_DMO_Winback_OBTM_UK_2019,[CAM0002279_DMO_WINBACK_OBTM_CAM0002279_OBTM_E...,TV Winback,OBTM Only
40,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback OBTM UK 2018,CAM0002279_DMO_Winback_OBTM_UK_2019,[CAM0002279_DMO_WINBACK_OBTM_CAM0002279_OBTM_E...,TV Winback,OBTM Only
41,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback OBTM UK 2018,CAM0002279_DMO_Winback_OBTM_UK_2019,[CAM0002279_DMO_WINBACK_OBTM_CAM0002279_OBTM_E...,TV Winback,OBTM Only
42,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback OBTM UK 2018,CAM0002279_DMO_Winback_OBTM_UK_2019,[CAM0002279_DMO_WINBACK_OBTM_CAM0002279_OBTM_E...,TV Winback,OBTM Only


In [286]:
provide_analysis_level_rules(campaigns_list,campaigns_list[['campaign_program','campaign_name']][0:1])

{'channel': True,
 'journey': True,
 'overall': True,
 'step': True,
 'variant': True}

In [299]:
create_right_granularity_level(campaigns_list,campaigns_list[['campaign_program','campaign_name']][13872:13873], 'step')[0]

,campaign_program,campaign_name,start_date,product,channel,journey,step
12,skyOptimusGetMore,Mobile,2018-11-30,Mobile,DM Only,CAM0002299 GETMORE Mobile Christmas DM OptIn U...,[DM_GETMORE_Mobile_Christmas_DM_OptIn_UK_2018]
9,skyOptimusGetMore,Mobile,2018-11-30,Mobile,EM Only,CAM0002299 GETMORE Mobile Christmas EM OptIn U...,[EM1_GETMORE_Mobile_Christmas_OptIn_UK_2018]
6,skyOptimusGetMore,Mobile,2018-11-30,Mobile,EM Only,CAM0002299 GETMORE Mobile Christmas EM OptIn U...,[EM2_GETMORE_Mobile_Christmas_OptIn_UK_2018]
0,skyOptimusGetMore,Mobile,2018-11-30,Mobile,EM&DM,CAM0002299 GETMORE Mobile Christmas EM_DM OptI...,[(EM) EM-DM_GETMORE_Mobile_Christmas_OptIn_UK_...
1,skyOptimusGetMore,Mobile,2018-11-30,Mobile,EM&DM,CAM0002299 GETMORE Mobile Christmas EM_DM OptI...,[(EM2) EM-DM_GETMORE_Mobile_Christmas_OptIn_UK...
7,skyOptimusGetMore,Mobile,2018-11-30,Mobile,EM&DM,CAM0002299 GETMORE Mobile Christmas EM_DM OptI...,[EM-DM_GETMORE_Mobile_Christmas_OptIn_UK_2018]


In [297]:
provide_analysis_level_rules(campaigns_list,campaigns_list[['campaign_program','campaign_name']][13872:13873])

3
1
3
3


{'channel': True,
 'journey': False,
 'overall': True,
 'step': True,
 'variant': True}

In [281]:
campaigns_list[campaigns_list.campaign_name=='Mobile'][0:1]

,campaign_program,campaign_name,start_date,journey,step,variant,product,channel
13872,skyOptimusGetMore,Mobile,2018-11-30,CAM0002299 GETMORE Mobile Christmas EM_DM OptI...,(EM) EM-DM_GETMORE_Mobile_Christmas_OptIn_UK_2018,CAM0002299_PRODUCT_UPGRADE_SKY_MOBILE_UK_1819_...,Mobile,EM&DM


We choose **Cinema Christmas** campaign. We extract the propmart attributes for customers involved in this campaign, and join.

In [703]:
PROPMART_FEAT = [ 'Base_Dt',
                 'account_number',
                 'DTV_Active',
                 'BB_Active',
                 'Sports_Active',
                 'Movies_Active',
                 'MS_Active',
                 'SGE_Active',
                 'HD_Active',
                 'Sky_Box_Sets_Active',
                 'UOD_Active',
                 'Age',
                 'h_equivalised_income_band',
                 'h_equivalised_income_value',
                 'h_residence_type',
                 'h_number_of_adults',
                 'h_number_of_children_in_hh',
                 'h_presence_of_child_aged_0_4',
                 'h_presence_of_child_aged_5_11',
                 'h_presence_of_child_aged_12_17',
                 'skyfibre_enabled',
                 'Broadband_Average_Demand',
                 'BT_Consumer_Market_Share',
                 'Sky_Consumer_Market_Share',
                 'Virgin_Consumer_Market_Share',
                 'TalkTalk_Consumer_Market_Share',
                 'Handset_Active',
                 'Cricket_Sports_Pref',
                 'Football_Sports_Pref',
                 'Rugby_Union_Sports_Pref',
                 'Rugby_League_Sports_Pref',
                 'F1_Sports_Pref',
                 'Darts_Sports_Pref',
                 'Tennis_Sports_Pref',
                 'Racing_Sports_Pref',
                 'Boxing_Sports_Pref',
                 'Golf_Sports_Pref',
                 'NFL_Sports_Pref',
                 'Wrestling_Sports_Pref',
                 'ESports_Sports_Pref',
                 'GAA_Sports_Pref',
                 'Motorsport_Sports_Pref',
                 'Cycling_Sports_Pref',
                 'Netball_Sports_Pref',
                 'SkyKids_Active',
                 'Up_Handset',
                 'Up_Mobile ']





Created BigQuery table: skyuk-uk-decis-models-01-dev.test_and_learn.prova2


In [109]:
PROJECT_ID_2 = 'skyuk-uk-decis-models-01-dev'
DATASET_ID_2 = 'uk_inp_prodrecs_ic'
PROP_MART_TABLE = 'Propensity_Model_Mart_Training'

In [363]:
create_campaign_analysis_table(PROJECT_ID_1,DATASET_ID_1,CAMPAING_TABLE_NAME,
                               OPTIMUS_COLS, 'skyOptimusGetMore', 'Cinema Christmas',
                               ['sale_target'],['CAM0002297_CONTENTEVENT_EMDMOPTIN_EM_CINEMA_UK_CAM0002297_EM_Cinema_Family_ContentEvent_EMDMoptin_Marketing_EM_1A20181207'],
                               'variant_label',
                               PROJECT_ID_2,DATASET_ID_2,PROP_MART_TABLE,
                               PROPMART_FEAT,'Base_Dt','2018-11-30',
                               '!=','Campaign control','prova2',step='EM1_Get_More_Cinema_Xmas_EM_DM_OptIn_UK_2018_EM')

Created BigQuery table: skyuk-uk-decis-models-01-dev.test_and_learn.prova2


Now we export into GCS

In [364]:
export_query_to_gcs(PROJECT_ID_1,DATASET_ID_1,'prova2','gs://'+BUCKET_ID+'/prova2.csv')

Exported skyuk-uk-decis-models-01-dev:test_and_learn.prova2 to gs://fci02/test_and_learn/prova2.csv


And then now we load the table

In [365]:
test_table = pd.read_csv('gs://'+BUCKET_ID+'/prova2.csv')
test_table.head()

,Base_Dt,account_number,TAs_in_last_14d,BB_Curr_Contract_Intended_End_Dt,DTV_Curr_Contract_Intnd_End_Dt,Curr_Offer_Intended_end_Dt_DTV,Curr_Offer_Intended_end_Dt_BB,BB_Last_Activation_Dt,DTV_Last_Activation_Dt,DTV_Active,...,Wrestling_Sports_Pref,ESports_Sports_Pref,GAA_Sports_Pref,Motorsport_Sports_Pref,Cycling_Sports_Pref,Netball_Sports_Pref,Up_Handset,Up_Mobile,L2_Campaign,target
0,2018-11-30,621376567338,0,2020-02-10,NaN,NaN,2020-02-10,2018-08-10,2010-04-02,1,...,0,0,0,0,0,0,0,0,Cinema Christmas,0
1,2018-11-30,630205519733,0,NaN,NaN,NaN,NaN,2016-02-15,2016-02-15,1,...,0,0,0,0,0,0,0,0,Cinema Christmas,0
2,2018-11-30,630153283472,0,NaN,NaN,NaN,NaN,2014-09-17,2012-11-21,1,...,0,0,0,0,0,0,0,0,Cinema Christmas,0
3,2018-11-30,621385652618,0,NaN,NaN,NaN,2019-07-13,2018-06-12,2010-05-05,1,...,0,0,0,0,0,0,0,0,Cinema Christmas,0
4,2018-11-30,622131952435,0,NaN,NaN,NaN,NaN,2017-03-06,2017-03-07,1,...,0,0,0,0,0,0,0,0,Cinema Christmas,0


In [366]:
test_table.shape

(63283, 55)

In [367]:
test_table.target.mean()

0.005499107185184014

## Data preprocessing

There is a minimal amount of data preprocessing we need to perform to ensure nothing will break during the analysis

In [18]:
def map_days_to_flag(row,type_of_flag):
    if(row<0):
        return 'out_of_{}'.format(type_of_flag)
    elif(row<90):
        return 'soon_out_of_{}'.format(type_of_flag)
    else:
        return 'in_{}'.format(type_of_flag)

In [19]:
def preprocess_data(input_frame,
                    mapping_dict,
                    not_to_use,
                    cut_threshold=.2):

    print('#################### DATA PREPROCESSING ####################')
    t0 = time.time()
    
    orig_len = input_frame.shape[0]
    
    print('1) Removing features with high percentage of nulls')
    to_exclude = ['BB_Curr_Contract_Intended_End_Dt', 'DTV_Curr_Contract_Intnd_End_Dt',
                 'Curr_Offer_Intended_end_Dt_DTV', 'Curr_Offer_Intended_end_Dt_BB',
                 'BB_Last_Activation_Dt','DTV_Last_Activation_Dt']
    to_check = [feature for feature in input_frame.columns if feature not in to_exclude]
    filterer = input_frame[to_check].isna().sum(axis=0)/len(input_frame)
    to_remove = filterer[filterer>=cut_threshold].index
    input_frame.drop(columns=to_remove,inplace=True)
    
    print('2) Features engineering')
    
    print('2a) Mappings')
    for feature, mapping in mapping_dict.items():
        if feature in input_frame.columns:
            input_frame[feature] = input_frame[feature].map(mapping)
    
    print('2b) Rounding income value')
    if  'h_equivalised_income_value' in input_frame.columns:
        input_frame['h_equivalised_income_value'] = (input_frame['h_equivalised_income_value'].dropna()/100).apply(lambda row:round(row))*100
        
    print('2c) Creating contract/offer status features')
    for feature, new_feature in zip(['BB_Curr_Contract_Intended_End_Dt', 'DTV_Curr_Contract_Intnd_End_Dt','Curr_Offer_Intended_end_Dt_DTV', 'Curr_Offer_Intended_end_Dt_BB'],
                                    ['BB_contract_status', 'DTV_contract_status', 'DTV_offer_status', 'BB_offer_status']):
        input_frame[new_feature]=(pd.to_datetime(input_frame[feature],errors='coerce')-pd.to_datetime(input_frame['Base_Dt'])).apply(lambda row: row.days).fillna(-1)
        input_frame[new_feature] = input_frame[new_feature].apply(lambda row: map_days_to_flag(row,new_feature.split('_')[1]))
        input_frame.drop(columns=[feature], inplace=True)
        
    print('2d) Creating number of days of tenure')
    input_frame['DTV_Last_Activation_days']=(pd.to_datetime(input_frame['DTV_Last_Activation_Dt'],errors='coerce')-pd.to_datetime(input_frame['Base_Dt'])).apply(lambda row: row.days)
    input_frame['BB_Last_Activation_days']=(pd.to_datetime(input_frame['BB_Last_Activation_Dt'],errors='coerce')-pd.to_datetime(input_frame['Base_Dt'])).apply(lambda row: row.days)
    input_frame['tenure_days']=-input_frame[['DTV_Last_Activation_days','BB_Last_Activation_days']].min(axis=1)
    input_frame.drop(columns=['DTV_Last_Activation_days','BB_Last_Activation_days','DTV_Last_Activation_Dt','BB_Last_Activation_Dt'], inplace=True)

    input_frame.dropna(inplace=True)
    final_len = input_frame.shape[0]
    print('     Number of rows lost: {}'.format(orig_len-final_len))
    print('     % of rows lost: {}%'.format(round((orig_len-final_len)*100/orig_len,2)))
    input_features = [column for column in input_frame.columns if column not in not_to_use]
    
    print('Data preprocessed in {} seconds'.format(round(time.time()-t0,3)))
    
    return input_features

### Preprocessing example

In [180]:
mapping_dict = {'h_presence_of_child_aged_0_4':{'No':0,'Yes':1},
                'h_presence_of_child_aged_5_11':{'No':0,'Yes':1},
                'h_presence_of_child_aged_12_17':{'No':0,'Yes':1}}

preprocess_data(test_table.copy(),mapping_dict,['Base_Dt', 'account_number','product','L2_Campaign','target'])

#################### DATA PREPROCESSING ####################
1) Removing features with high percentage of nulls
2) Features engineering
2a) Mappings
2b) Rounding income value
2c) Creating contract/offer status features
2d) Creating number of days of tenure
     Number of rows lost: 7676
     % of rows lost: 9.31%
Data preprocessed in 5.292 seconds


['TAs_in_last_14d',
 'DTV_Active',
 'BB_Active',
 'Sports_Active',
 'Movies_Active',
 'MS_Active',
 'SGE_Active',
 'HD_Active',
 'Sky_Box_Sets_Active',
 'UOD_Active',
 'SkyKids_Active',
 'Handset_Active',
 'skyfibre_enabled',
 'Age',
 'h_equivalised_income_value',
 'h_residence_type',
 'h_number_of_adults',
 'h_number_of_children_in_hh',
 'h_presence_of_child_aged_0_4',
 'h_presence_of_child_aged_5_11',
 'h_presence_of_child_aged_12_17',
 'BT_Consumer_Market_Share',
 'Sky_Consumer_Market_Share',
 'Virgin_Consumer_Market_Share',
 'TalkTalk_Consumer_Market_Share',
 'Cricket_Sports_Pref',
 'Football_Sports_Pref',
 'Rugby_Union_Sports_Pref',
 'Rugby_League_Sports_Pref',
 'F1_Sports_Pref',
 'Darts_Sports_Pref',
 'Tennis_Sports_Pref',
 'Racing_Sports_Pref',
 'Boxing_Sports_Pref',
 'Golf_Sports_Pref',
 'NFL_Sports_Pref',
 'Wrestling_Sports_Pref',
 'ESports_Sports_Pref',
 'GAA_Sports_Pref',
 'Motorsport_Sports_Pref',
 'Cycling_Sports_Pref',
 'Netball_Sports_Pref',
 'Up_Handset',
 'Up_Mobile',


## Rules creation utils

In [20]:
def is_binary(series):
    return sorted(series.unique()) == [0, 1]

In [21]:
def convert_to_binary(array):
    return (array*2**(np.arange(len(array)))).sum()

In [22]:
def print_interval(list_of_ineq):
    lower = np.array(list_of_ineq)[np.char.startswith(list_of_ineq,'>')][0]
    upper = np.array(list_of_ineq)[np.char.startswith(list_of_ineq,'<=')][0]

    lower = lower.split(sep=' ')[-1]
    upper = upper.split(sep=' ')[-1]

    return [lower + ' < X <= '+upper]

In [23]:
def clean_rules(dicto):
    for key in dicto.keys():
        
        # basic cleaning: if we do not have rule then we state there is no rule
        if not len(dicto[key]):
            dicto[key].append(None)
        if len(dicto[key])>1:
            if np.char.startswith(dicto[key],'<=').all() or np.char.startswith(dicto[key],'>').all():
                # case 1: only '<=' or '>' inequalities
                dicto[key] = ['X '+dicto[key][-1]]
            elif (not np.char.startswith(dicto[key],'<=').all()) and len(dicto[key])==2:
                # case 2: just one >= and one < inequalities
                dicto[key] = print_interval(dicto[key])
            elif (not np.char.startswith(dicto[key],'<=').all()) and len(dicto[key])>2:
                # case 3: mixed case
                string_array = np.array(dicto[key])
                reduced_list = [string_array[np.char.startswith(string_array,'<=')][-1],
                                string_array[np.char.startswith(string_array,'>')][-1]]
                dicto[key] = print_interval(reduced_list)

In [24]:
def compute_uplift_table(treatment_set, control_set, 
                         best_features, target,
                         learner,
                         significance=0.05):
    
    list_of_frames = []
    
    for dataset,label in zip([treatment_set, control_set],
                             ['treatment','control']):
        
        X = pd.get_dummies(dataset,drop_first=True)[best_features].values
        y = dataset[target].values

        learner_paths = np.array((learner.decision_path(X).todense()))
        learner_paths = np.apply_along_axis(convert_to_binary, 1, learner_paths)
        
        data = np.hstack((learner_paths.reshape(-1,1),y.reshape(-1,1)))

        out_frame = pd.DataFrame(data=data,columns=['path_id','target'],dtype=float)
        out_frame = out_frame.groupby(by=['path_id']).agg(**{label+'_sample_size': pd.NamedAgg(column='target', aggfunc='count'),
                                                             label+'_converters': pd.NamedAgg(column='target', aggfunc='sum'),
                                                             label+'_conversion': pd.NamedAgg(column='target', aggfunc='mean')})
        
        list_of_frames.append(out_frame)

    # If some of the treatment rules are not covered in control then we ignore them
    lift_frame = pd.merge(list_of_frames[0],list_of_frames[1],on=['path_id'])
        
    lift_frame['uplift'] = lift_frame['treatment_conversion']-lift_frame['control_conversion']
    lift_frame.sort_values(by=['uplift'],ascending=False, inplace=True)
    
    lift_frame['p_value']=lift_frame.apply(lambda row: proportions_chisquare(row[['treatment_converters','control_converters']],
                                                                             row[['treatment_sample_size','control_sample_size']])[1],axis=1)
    lift_frame['significative'] = 'No'
    lift_frame.loc[lift_frame['p_value']<significance,'significative'] = 'Yes'
    
    
    return  lift_frame

In [25]:
def construct_rules(treatment_frame, estimator, features, mapping, lift_frame, print_rules=False):

    X = pd.get_dummies(treatment_frame[features],drop_first=True).values
        
    learner_paths = np.array((estimator.decision_path(X).todense()))
    learner_paths = np.apply_along_axis(convert_to_binary, 1, learner_paths)
        
    data = np.hstack((X,learner_paths.reshape(-1,1)))
    tmp_frame = pd.DataFrame(data=data,columns=list(features)+['path_id'],dtype=float)
    tmp_frame.drop_duplicates(subset=['path_id'],inplace=True)
    tmp_frame.set_index(['path_id'],inplace=True)
    unique_paths = tmp_frame.loc[lift_frame.index,features].values
    
    for feature in features:
        lift_frame[feature]=np.nan
    
    rules = {feature:[] for feature in features}
    
    n_nodes = estimator.tree_.node_count
    children_left = estimator.tree_.children_left
    children_right = estimator.tree_.children_right
    feature = estimator.tree_.feature
    threshold = estimator.tree_.threshold

    node_indicator = estimator.decision_path(unique_paths)
    leave_id = estimator.apply(unique_paths)
    

    
    for sample_id in range(len(unique_paths)):
        node_index = node_indicator.indices[node_indicator.indptr[sample_id]:
                                            node_indicator.indptr[sample_id + 1]]

        #print('Rules used to predict sample %s: ' % sample_id)
        tmp_rules = {feature:[] for feature in features}
        for node_id in node_index:
            if leave_id[sample_id] == node_id:
                continue

            if (unique_paths[sample_id, feature[node_id]] <= threshold[node_id]):
                threshold_sign = "<="
                value = int(round(threshold[node_id],0))
            else:
                threshold_sign = ">"
                value = int(threshold[node_id])

            if mapping[features[feature[node_id]]]=='binary':
                tmp_rules[features[feature[node_id]]].append('Yes' if unique_paths[sample_id, feature[node_id]] else 'No')
            else:
                tmp_rules[features[feature[node_id]]].append(threshold_sign+' {}'.format(value))

            """if print_rules:
                print("%s (= %s) %s %s"
                          % (features[feature[node_id]],
                             unique_paths[sample_id, feature[node_id]],
                             threshold_sign,
                             value))"""
        clean_rules(tmp_rules)     

        for feature_name in features:
            rules[feature_name].append(tmp_rules[feature_name][0])

    for feature in features:
        lift_frame[feature] = rules[feature]
    return lift_frame

## Tree rules generation

We now build a class that trains a Random Forest classifier on the treatment set, identifies the top three relevant features. The latter are then fed into a Decision Tree classifier so that hihg level rules can be learnt. Both the forest and the tree are kept *simple*, i.e. we will try to build them not too much deep.

In [124]:
def create_logger(log_file_name=None):
    
    for handler in logging.root.handlers[:]:
        logging.root.removeHandler(handler)
    
    if log_file_name is not None:
        logging.basicConfig(filename=log_file_name, filemode='w',
                         format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S')
    else:
        logging.basicConfig(format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S')
    logger = logging.getLogger('TreeRulesExtractor')
    logger.setLevel(logging.INFO)
    print = logger.info
    return logger

In [27]:
def compute_auc_metrics(learner,
                        X_train,X_test,
                        y_train,y_test):

    train_proba = learner.predict_proba(X_train)
    test_proba= learner.predict_proba(X_test)

    fpr = {}
    tpr = {}

    summary = {}

    for proba, target, label in zip([train_proba, test_proba],
                                    [y_train,y_test],
                                    ['train','test']):
        fpr[label], tpr[label], thresholds = roc_curve(target, proba[:, 1])
        summary[label] = auc(fpr[label], tpr[label])

    return summary

In [28]:
class TreeRulesExtractor(object):
    
    def __init__(self,
                 target_feature,
                 input_features,
                 n_estimators,
                 min_samples_leaf ,
                 max_depth,
                 max_allowed_depth,
                 min_allowed_node_size,
                 logger,
                 njobs=-1,
                 n_best_features=3):
    
        self.target_feature = target_feature
        self.input_features = input_features
        self.n_estimators = n_estimators
        self.min_samples_leaf = min_samples_leaf
        self.max_depth = max_depth
        self.max_allowed_depth = max_allowed_depth
        self.min_allowed_node_size = min_allowed_node_size
        self.n_best_features = n_best_features
        self.njobs = njobs
        self.best_features = []
        
        if self.max_allowed_depth not in self.max_depth:
            self.max_depth = list(self.max_depth)+[self.max_allowed_depth]
            
        if self.min_allowed_node_size not in self.min_samples_leaf:
            self.min_samples_leaf = list(self.min_samples_leaf)+[self.min_allowed_node_size]
        
        self.logger = logger
                
    def prepare_data(self,
                     treatment_frame,
                     test_size,
                     rstate):
        inputs = pd.get_dummies(treatment_frame[self.input_features], drop_first=True)
        self.all_features = inputs.columns
        
        X = inputs.values
        y  = treatment_frame[self.target_feature].values

        X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                            test_size=test_size, 
                                                            random_state=rstate,
                                                            stratify=y)

        return X_train, X_test, y_train, y_test

    def forest_tuning(self,
                      treatment_frame,
                      test_size,
                      rstate,
                      tuning=True,
                      display_frame=False):
         
        params = {'n_estimators':self.n_estimators,
                  'min_samples_leaf':self.min_samples_leaf,
                  'max_depth':self.max_depth}

        self.logger.log(20,'Random Forest hyperparameters tuning')
        rf_learner = RandomForestClassifier(n_jobs=self.njobs)
        
        X_train, X_test, y_train, y_test = self.prepare_data(treatment_frame,
                                                             test_size,
                                                             rstate)
        
        X_full = np.vstack((X_train,X_test))
        y_full = np.hstack((y_train,y_test))

        train_part = np.zeros(len(X_train))-1
        test_part = np.ones(len(X_test))
        test_fold = np.hstack((train_part,test_part))
        splitter = PredefinedSplit(test_fold)
        
        if tuning:
            self.logger.info('Optimal Random Forest tuning')
            searcher = GridSearchCV(estimator=rf_learner, 
                                    param_grid=params, 
                                    scoring='roc_auc', 
                                    n_jobs=1,
                                    refit=True, cv=splitter,
                                    verbose=2, 
                                    pre_dispatch=1,
                                    return_train_score=False)
            searcher.fit(X_full,y_full)
            optimal_forest_pms = searcher.best_params_

            self.logger.info('Optimal Random Forest training')
            optimal_forest = RandomForestClassifier(n_estimators=optimal_forest_pms['n_estimators'],
                                        max_depth=optimal_forest_pms['max_depth'],
                                        min_samples_leaf=optimal_forest_pms['min_samples_leaf'],
                                        verbose=1,
                                        random_state=rstate,
                                        n_jobs=self.njobs)
            optimal_forest.fit(X_train,y_train)
        else:
            optimal_forest_pms={'n_estimators':max(self.n_estimators),
                                'min_samples_leaf':self.min_allowed_node_size,
                                'max_depth':self.max_allowed_depth}
            
        self.minimal_forest_pms = {'n_estimators':optimal_forest_pms['n_estimators'],
                                   'min_samples_leaf':self.min_allowed_node_size,
                                   'max_depth':self.max_allowed_depth}
        
        
        if self.min_allowed_node_size < optimal_forest_pms['min_samples_leaf']:
            self.minimal_forest_pms['min_samples_leaf'] = optimal_forest_pms['min_samples_leaf']
        if self.max_allowed_depth > optimal_forest_pms['max_depth']:
            self.minimal_forest_pms['max_depth'] = optimal_forest_pms['max_depth']
        
        self.logger.info('Minimal Random Forest training')
        minimal_forest = RandomForestClassifier(n_estimators=self.minimal_forest_pms['n_estimators'],
                                                max_depth=self.minimal_forest_pms['max_depth'],
                                                min_samples_leaf=self.minimal_forest_pms['min_samples_leaf'],
                                                verbose=1,
                                                random_state=rstate,
                                                n_jobs=self.njobs)
        minimal_forest.fit(X_train,y_train)
                
        self.logger.info('Best top {} features extraction. Single tree performance analysis'.format(self.n_best_features))
        best_feat_index = np.argsort(-minimal_forest.feature_importances_)[:self.n_best_features]
        minimal_tree = DecisionTreeClassifier(max_depth=self.minimal_forest_pms['max_depth'],
                                              min_samples_leaf=self.minimal_forest_pms['min_samples_leaf'])
        self.forest_summary = {}
        
        self.logger.info('Collecting analysis of performances')     
        if tuning:
            list_of_learners=[optimal_forest,minimal_forest]
            list_of_names=['optimal_forest','minimal_forest']
        else:
            list_of_learners=[minimal_forest]
            list_of_names=['minimal_forest']
            
        for forest, label in zip(list_of_learners,
                                 list_of_names):    
            summary = compute_auc_metrics(forest, X_train,X_test, y_train,y_test)
            summary.update({'n_estimators':forest.n_estimators,
                            'min_samples_leaf':forest.min_samples_leaf,
                            'max_depth':forest.max_depth})     
            self.forest_summary[label] = summary

        minimal_tree.fit(X_train[:,best_feat_index],y_train)
        summary = compute_auc_metrics(minimal_tree, X_train[:,best_feat_index],X_test[:,best_feat_index], y_train,y_test)
        summary.update({'n_estimators':1,
                        'min_samples_leaf':minimal_tree.min_samples_leaf,
                        'max_depth':minimal_tree.max_depth}) 
        self.forest_summary['minimal_tree'] = summary
        
        if display_frame:
            display(pd.DataFrame(self.forest_summary))
            
        self.best_features = list(pd.DataFrame(data=minimal_forest.feature_importances_,
                                  index=self.all_features).sort_values(by=0,ascending=False).index[:self.n_best_features])
       
    def tree_rules_generation(self,treatment_frame,control_frame,
                              significance=0.05,print_tree=False):
        
        if not len(self.best_features):
            self.logger.error('No best {} features identified. You first have to run "forest_tuning" method'.format(self.n_best_features))
        
        X_treat_set = pd.get_dummies(treatment_frame, drop_first=True)[self.best_features+[self.target_feature]]
        
        X_ctr_set = pd.get_dummies(control_frame, drop_first=True)[self.best_features+[self.target_feature]]
        
        ruler = DecisionTreeClassifier(max_depth=self.minimal_forest_pms['max_depth'],
                                       min_samples_leaf=self.minimal_forest_pms['min_samples_leaf'],
                                      random_state=42)
        ruler.fit(X_treat_set[self.best_features].values,X_treat_set[self.target_feature].values)
        
        if print_tree:
            export_graphviz(ruler, 
            out_file='./tree.dot', 
            feature_names = self.best_features,
            class_names = None,
            rounded = True, proportion = False, 
            precision = 2, filled = True,special_characters=True)

            call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=800'])
            display(Image(filename = 'tree.png'))
        
        uplift_table = compute_uplift_table(X_treat_set, X_ctr_set, 
                                            self.best_features, self.target_feature,
                                            ruler,
                                            significance=significance)
        
        
        # Build mapping of the features
        mapping = {}
        for feature in self.best_features:
            if is_binary(X_treat_set[feature]):
                mapping[feature]='binary'
            else:
                mapping[feature]='continuos'
            
        rules_table = construct_rules(X_treat_set, ruler, self.best_features, 
                                      mapping, uplift_table, print_rules=print_tree)
        return rules_table

### Rules generation example

#### creation of treatment table

In [26]:
create_campaign_analysis_table(PROJECT_ID_1,DATASET_ID_1,CAMPAING_TABLE_NAME,
                               OPTIMUS_COLS, 'skyOptimusGetMore', 'Cinema Christmas','Cinema',
                               PROJECT_ID_2,DATASET_ID_2,PROP_MART_TABLE,
                               PROPMART_FEAT,'Base_Dt','2018-11-30',
                               '=','Fallow Group','temp_table_tr')
export_query_to_gcs(PROJECT_ID_1,DATASET_ID_1,'temp_table_tr','gs://'+BUCKET_ID+'/Cinema_Christmas_treatment.csv')
treatment_table = pd.read_csv('gs://'+BUCKET_ID+'/Cinema_Christmas_treatment.csv')

NameError: name 'PROJECT_ID_2' is not defined

#### creation of control table

In [127]:
create_campaign_analysis_table(PROJECT_ID_1,DATASET_ID_1,CAMPAING_TABLE_NAME,
                               OPTIMUS_COLS, 'skyOptimusGetMore', 'Cinema Christmas','Cinema',
                               PROJECT_ID_2,DATASET_ID_2,PROP_MART_TABLE,
                               PROPMART_FEAT,'Base_Dt','2018-11-30',
                               '!=','Fallow Group','temp_table_ctr')
export_query_to_gcs(PROJECT_ID_1,DATASET_ID_1,'temp_table_ctr','gs://'+BUCKET_ID+'/Cinema_Christmas_control.csv')
control_table = pd.read_csv('gs://'+BUCKET_ID+'/Cinema_Christmas_control.csv')

Created BigQuery table: skyuk-uk-decis-models-01-dev.test_and_learn.temp_table_ctr
Exported skyuk-uk-decis-models-01-dev:test_and_learn.temp_table_ctr to gs://fci02/test_and_learn/Cinema_Christmas_control.csv


#### rules

In [128]:
treatment_table.head()

,Base_Dt,account_number,DTV_Active,BB_Active,Sports_Active,Movies_Active,MS_Active,SGE_Active,HD_Active,Sky_Box_Sets_Active,...,GAA_Sports_Pref,Motorsport_Sports_Pref,Cycling_Sports_Pref,Netball_Sports_Pref,SkyKids_Active,Up_Handset,Up_Mobile,product,L2_Campaign,target
0,2018-11-30,620023990489,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,Cinema,Cinema Christmas,0
1,2018-11-30,400033078637,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,Cinema,Cinema Christmas,0
2,2018-11-30,400035194309,1,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,Cinema,Cinema Christmas,0
3,2018-11-30,400031413638,1,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,Cinema,Cinema Christmas,0
4,2018-11-30,630216314629,1,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,Cinema,Cinema Christmas,0


In [129]:
treatment_table.shape

(1040362, 50)

In [130]:
control_table.head()

,Base_Dt,account_number,DTV_Active,BB_Active,Sports_Active,Movies_Active,MS_Active,SGE_Active,HD_Active,Sky_Box_Sets_Active,...,GAA_Sports_Pref,Motorsport_Sports_Pref,Cycling_Sports_Pref,Netball_Sports_Pref,SkyKids_Active,Up_Handset,Up_Mobile,product,L2_Campaign,target
0,2018-11-30,620051243249,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,Cinema,Cinema Christmas,0
1,2018-11-30,220015765740,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,Cinema,Cinema Christmas,1
2,2018-11-30,622978421080,1,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,Cinema,Cinema Christmas,0
3,2018-11-30,623297576539,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,Cinema,Cinema Christmas,0
4,2018-11-30,621501264918,1,1,1,0,0,1,1,0,...,0,0,0,0,0,0,0,Cinema,Cinema Christmas,0


In [131]:
control_table.shape

(25910, 50)

In [132]:
mapping_dict = {'h_presence_of_child_aged_0_4':{'No':0,'Yes':1},
                'h_presence_of_child_aged_5_11':{'No':0,'Yes':1},
                'h_presence_of_child_aged_12_17':{'No':0,'Yes':1},}

preprocess_data(treatment_table,mapping_dict,['Base_Dt', 'account_number','product','L2_Campaign','target'])
preprocess_data(control_table,mapping_dict,['Base_Dt', 'account_number','product','L2_Campaign','target'])

#################### DATA PREPROCESSING ####################
1) Removing features with high percentage of nulls
     Number of rows lost: 164707
     % of rows lost: 15.83%
2) Features engineering
Data preprocessed in 2.405 seconds
#################### DATA PREPROCESSING ####################
1) Removing features with high percentage of nulls
     Number of rows lost: 2493
     % of rows lost: 9.62%
2) Features engineering
Data preprocessed in 0.067 seconds


['DTV_Active',
 'BB_Active',
 'Sports_Active',
 'Movies_Active',
 'MS_Active',
 'SGE_Active',
 'HD_Active',
 'Sky_Box_Sets_Active',
 'UOD_Active',
 'Age',
 'h_equivalised_income_band',
 'h_equivalised_income_value',
 'h_residence_type',
 'h_number_of_adults',
 'h_number_of_children_in_hh',
 'h_presence_of_child_aged_0_4',
 'h_presence_of_child_aged_5_11',
 'h_presence_of_child_aged_12_17',
 'skyfibre_enabled',
 'BT_Consumer_Market_Share',
 'Sky_Consumer_Market_Share',
 'Virgin_Consumer_Market_Share',
 'TalkTalk_Consumer_Market_Share',
 'Handset_Active',
 'Cricket_Sports_Pref',
 'Football_Sports_Pref',
 'Rugby_Union_Sports_Pref',
 'Rugby_League_Sports_Pref',
 'F1_Sports_Pref',
 'Darts_Sports_Pref',
 'Tennis_Sports_Pref',
 'Racing_Sports_Pref',
 'Boxing_Sports_Pref',
 'Golf_Sports_Pref',
 'NFL_Sports_Pref',
 'Wrestling_Sports_Pref',
 'ESports_Sports_Pref',
 'GAA_Sports_Pref',
 'Motorsport_Sports_Pref',
 'Cycling_Sports_Pref',
 'Netball_Sports_Pref',
 'SkyKids_Active',
 'Up_Handset',
 'Up

In [133]:
input_feats = ['DTV_Active', 'BB_Active', 'Sports_Active',
       'Movies_Active', 'MS_Active', 'SGE_Active', 'HD_Active',
       'Sky_Box_Sets_Active', 'UOD_Active', 'Age', 
       'h_equivalised_income_value', 'h_residence_type', 'h_number_of_adults',
       'h_number_of_children_in_hh', 'h_presence_of_child_aged_0_4',
       'h_presence_of_child_aged_5_11', 'h_presence_of_child_aged_12_17',
       'skyfibre_enabled', 'BT_Consumer_Market_Share',
       'Sky_Consumer_Market_Share', 'Virgin_Consumer_Market_Share',
       'TalkTalk_Consumer_Market_Share', 'Handset_Active',
       'Cricket_Sports_Pref', 'Football_Sports_Pref',
       'Rugby_Union_Sports_Pref', 'Rugby_League_Sports_Pref', 'F1_Sports_Pref',
       'Darts_Sports_Pref', 'Tennis_Sports_Pref', 'Racing_Sports_Pref',
       'Boxing_Sports_Pref', 'Golf_Sports_Pref', 'NFL_Sports_Pref',
       'Wrestling_Sports_Pref', 'ESports_Sports_Pref', 'GAA_Sports_Pref',
       'Motorsport_Sports_Pref', 'Cycling_Sports_Pref', 'Netball_Sports_Pref',
       'SkyKids_Active', 'Up_Handset', 'Up_Mobile']

In [134]:
rules_generator = TreeRulesExtractor('target',
                                     input_feats,
                                     [00],
                                     [5000],
                                     [7],
                                      7,
                                      5000,
                                      create_logger(),
                                      12)

In [135]:
rules_generator.forest_tuning(treatment_table,
                              .3,
                              42,
                              display_frame=True)

22-Oct-19 14:15:11 - Random Forest hyperparameters tuning


Fitting 1 folds for each of 1 candidates, totalling 1 fits
[CV] n_estimators=300, min_samples_leaf=5000, max_depth=7 ............


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  n_estimators=300, min_samples_leaf=5000, max_depth=7, total=  30.0s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   30.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   30.5s finished
22-Oct-19 14:16:26 - Optimal Random Forest training
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    2.1s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:   11.4s
[Parallel(n_jobs=12)]: Done 300 out of 300 | elapsed:   19.6s finished
22-Oct-19 14:16:47 - Minimal Random Forest training
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    2.0s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:   11.8s
[Parallel(n_jobs=12)]: Done 300 out of 300 | elapsed:   19.9s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.2s
[Parallel(n_jobs=12)]: Done 176 tasks      | ela

,minimal_forest,minimal_tree,optimal_forest
max_depth,7.000000,7.000000,7.000000
min_samples_leaf,5000.000000,5000.000000,5000.000000
n_estimators,300.000000,1.000000,300.000000
test,0.596340,0.572135,0.596956
train,0.602632,0.575455,0.602999


In [136]:
rules_generator.best_features

['SkyKids_Active', 'Sky_Box_Sets_Active', 'SGE_Active']

In [137]:
rules_generator.tree_rules_generation(treatment_table,control_table,
                                      significance=0.05,print_tree=True)

,treatment_conversion,treatment_converters,treatment_sample_size,control_conversion,control_converters,control_sample_size,uplift,p_value,significative,SkyKids_Active,Sky_Box_Sets_Active,SGE_Active
path_id,,,,,,,,,,,,
163.0,0.019804,504.0,25450,0.010955,7.0,639,0.008849,0.110869,No,Yes,No,Yes
99.0,0.014236,4011.0,281759,0.010250,78.0,7610,0.003986,0.003650,Yes,No,No,Yes
12545.0,0.021033,537.0,25531,0.018405,12.0,652,0.002628,0.643689,No,No,Yes,Yes
15.0,0.009804,4384.0,447143,0.007320,88.0,12022,0.002485,0.006193,Yes,No,No,No
1793.0,0.017162,511.0,29775,0.016169,13.0,804,0.000993,0.830497,No,No,Yes,No
20737.0,0.025723,444.0,17261,0.025822,11.0,426,-0.000099,0.989840,No,Yes,Yes,Yes
23.0,0.017196,600.0,34891,0.019715,18.0,913,-0.002519,0.564030,No,Yes,No,No
2817.0,0.020007,277.0,13845,0.028490,10.0,351,-0.008483,0.264786,No,Yes,Yes,No


In [138]:
campaign_rules = rules_generator.tree_rules_generation(treatment_table,control_table,
                                                       significance=0.05,print_tree=False)

## Final formatting

We then add format and push the table into GCP

In [29]:
def output_formatting(table, optimus_columns, subset_row,granularity_level):
    if granularity_level=='overall':
        table[optimus_columns['channel_label']]='all'
        table[optimus_columns['journey_label']]='all'
        table[optimus_columns['step_label']]='all'
        table[optimus_columns['variant_label']]='all'
    elif granularity_level=='channel':
        table[optimus_columns['channel_label']]=subset_row['channel']
        table[optimus_columns['journey_label']]='NA'
        table[optimus_columns['step_label']]='NA'
        table[optimus_columns['variant_label']]='NA'
    elif granularity_level=='journey':
        table[optimus_columns['channel_label']]=subset_row['channel']
        table[optimus_columns['journey_label']]=subset_row['journey'][0]            
        table[optimus_columns['step_label']]='NA'
        table[optimus_columns['variant_label']]='NA'
    elif granularity_level=='step':
        table[optimus_columns['channel_label']]=subset_row['channel']
        table[optimus_columns['journey_label']]=subset_row['journey']       
        table[optimus_columns['step_label']]=subset_row['step'][0]
        table[optimus_columns['variant_label']]='NA'
    elif granularity_level=='variant':
        table[optimus_columns['channel_label']]=subset_row['channel']
        table[optimus_columns['journey_label']]=subset_row['journey']       
        table[optimus_columns['step_label']]=subset_row['step']
        table[optimus_columns['variant_label']]=subset_row['variant'][0]

In [30]:
def format_and_push_table(project_id: str, 
                          dataset_id: str,
                          table_id: str,
                          table_to_push,
                          rules_features,
                          subset_row,
                          granularity_level,                          
                          optimus_columns,
                          logger,
                          missing_control,
                          format_table=False):
    
    if format_table:
        print(rules_features)
        logger.info('Formatting rules table')
        table = table_to_push.reset_index(drop=True)

        for feature,feature_name in zip([subset_row['campaign_program'], subset_row['campaign_name'], subset_row['product'], subset_row['start_date']],
                                         [optimus_columns['program_label'],optimus_columns['campaign_label'], optimus_columns['product_label'], optimus_columns['start_date_label']]):
            table[feature_name] = feature

        table['analysis_level']=granularity_level    
        
        output_formatting(table, optimus_columns, subset_row,granularity_level)
            
        new_cols_1 = []
        new_cols_2 = []
        for index,feature in enumerate(rules_features,1):
            table['best_feature_{}'.format(index)] = feature
            table['best_feature_{}_rule'.format(index)] = table[feature]

            new_cols_1.append('best_feature_{}'.format(index))
            new_cols_2.append('best_feature_{}_rule'.format(index))


        table['uplift_ranking'] = table.index.values+1
        table.sort_values(by=['treatment_conversion'],inplace=True, ascending=False)
        table['treatment_conversion_ranking'] = np.arange(1,len(table)+1)

        table.drop(columns=['treatment_converters','control_converters'],inplace=True)

        core_to_show = [optimus_columns['program_label'],
                        optimus_columns['campaign_label'], 
                        optimus_columns['start_date_label'],
                        optimus_columns['product_label'],
                        'analysis_level',
                        optimus_columns['channel_label'],
                        optimus_columns['journey_label'],
                        optimus_columns['step_label'],
                        optimus_columns['variant_label']]+new_cols_1+new_cols_2

        to_show = core_to_show + ['treatment_sample_size','control_sample_size',
                                  'treatment_conversion','control_conversion',
                                  'uplift','p_value','significative',
                                  'uplift_ranking','treatment_conversion_ranking']
        # if control is missing, we amend the table output
        if missing_control:
            table['control_sample_size'] = 0
            table['control_conversion'] = 0
            table['uplift'] = table['treatment_conversion']
            table['uplift_ranking'] = table['treatment_conversion_ranking']
            table['p_value'] = np.nan
            table['significative'] = np.nan
            
        return table.sort_values(by=['uplift_ranking'])[to_show]
    else:
        pass
    
        logger.info('Saving table to GCP')
        schema = []
        for column in table_to_push.columns:
            val_type = str(type(table_to_push[column].values[0]))
            if 'int' in val_type:
                col_type = 'INT64'
            elif 'float' in val_type:
                col_type = 'FLOAT64'
            elif 'str' in val_type:
                col_type = 'STRING'
            elif 'datetime' in val_type:
                col_type = 'DATE'
            
            schema.append(bigquery.SchemaField(column, col_type, mode="REQUIRED"))
        job_config = bigquery.LoadJobConfig(schema=schema, write_disposition="WRITE_APPEND")

        # Create or overwrite the existing table if it exists
        table_id = '{}.{}.{}'.format(project_id,dataset_id,table_id)
        bq = bigquery.Client(project=project_id)
        job = bq.load_table_from_dataframe(table_to_push,
                                           table_id,
                                           job_config=job_config,
                                           location="EU")
        job.result() # Waits for table load to complete. 

In [31]:
def format_performance_frame(performance_frame,subset_row,granularity_level):
    
    for feature,feature_name in zip(['campaign_name','product','start_date','campaign_program'],
                                    [subset_row.loc['campaign_name'], subset_row.loc['product'], 
                                     subset_row.loc['start_date'],subset_row.loc['campaign_program']]):
        performance_frame[feature] = feature_name
    
    if granularity_level=='overall':
        performance_frame['channel']='all'
        performance_frame['journey']='all'
        performance_frame['step']='all'
        performance_frame['variant']='all'
    elif granularity_level=='channel':
        performance_frame['channel']=subset_row.loc['channel']
        performance_frame['journey']='NA'
        performance_frame['step']='NA'
        performance_frame['variant']='NA'
    elif granularity_level=='journey':
        performance_frame['channel']=subset_row.loc['channel']
        performance_frame['journey']=subset_row.loc['journey'][0]
        performance_frame['step']='NA'
        performance_frame['variant']='NA'
    elif granularity_level=='journey':
        performance_frame['channel']=subset_row.loc['channel']
        performance_frame['journey']=subset_row.loc['journey']
        performance_frame['step']=performance_frame['step'][0]
        performance_frame['variant']='NA'
    elif granularity_level=='journey':
        performance_frame['channel']=subset_row.loc['channel']
        performance_frame['journey']=subset_row.loc['journey']
        performance_frame['step']=performance_frame['step']
        performance_frame['variant']=performance_frame['variant'][0]
    return performance_frame

In [32]:
def batch_save(idx_value,
               bucket_id,
               list_of_performances,
               list_of_tables,
               logger,
               campaign_data_paths,
               out_table_name):
    logger.info('Saving performance information')
    time_of_day = datetime.datetime.now()
    perf_file_name = 'campaign_analysis_perf_{}-{}-{}_check_point_{}.csv'.format(time_of_day.year,
                                                                                 time_of_day.month,
                                                                                 time_of_day.day,
                                                                                 idx_value)
    perf_path ='gs://'+os.path.join(bucket_id,perf_file_name)
    list_of_performances = pd.concat(list_of_performances,axis=0)
    list_of_performances.to_csv(perf_path)
    logger.info('Performance information saved in {}'.format(perf_path))

    # collect all campaign infos within a unique dataframe
    # push to GCP
    logger.info('Collecting results, saving to GCP')
    list_of_tables = pd.concat(list_of_tables,axis=0)
    format_and_push_table(campaign_data_paths['project_id'], campaign_data_paths['dataset_id'],out_table_name,list_of_tables,
                          None, None, None,None,logger, None, False)

### Example of output table

We can inspect the table we are outputing

In [233]:
logger = create_logger(None)
format_and_push_table(PROJECT_ID_1,DATASET_ID_1,'lightweight_test',campaign_rules,
                      rules_generator.best_features,
                     'Cinema Christmas',
                     'Cinema',
                     '12-11-2018',
                     'GetMOre',
                     'Email',
                     OPTIMUS_COLS,logger,
                     missing_control=False,
                     format_table=True)

23-Oct-19 15:15:25 - Formatting rules table


,L1_programme,L2_Campaign,CAMPAIGN_REPORTING_START_DATE,product,CHANNEL,best_feature_1,best_feature_2,best_feature_3,best_feature_1_rule,best_feature_2_rule,best_feature_3_rule,treatment_sample_size,control_sample_size,treatment_conversion,control_conversion,uplift,p_value,significative,uplift_ranking,treatment_conversion_ranking
0,GetMOre,Cinema Christmas,12-11-2018,Cinema,Email,SkyKids_Active,Sky_Box_Sets_Active,SGE_Active,Yes,No,Yes,25450,639,0.019804,0.010955,0.008849,0.110869,No,1,4
1,GetMOre,Cinema Christmas,12-11-2018,Cinema,Email,SkyKids_Active,Sky_Box_Sets_Active,SGE_Active,No,No,Yes,281759,7610,0.014236,0.010250,0.003986,0.003650,Yes,2,7
2,GetMOre,Cinema Christmas,12-11-2018,Cinema,Email,SkyKids_Active,Sky_Box_Sets_Active,SGE_Active,No,Yes,Yes,25531,652,0.021033,0.018405,0.002628,0.643689,No,3,2
3,GetMOre,Cinema Christmas,12-11-2018,Cinema,Email,SkyKids_Active,Sky_Box_Sets_Active,SGE_Active,No,No,No,447143,12022,0.009804,0.007320,0.002485,0.006193,Yes,4,8
4,GetMOre,Cinema Christmas,12-11-2018,Cinema,Email,SkyKids_Active,Sky_Box_Sets_Active,SGE_Active,No,Yes,No,29775,804,0.017162,0.016169,0.000993,0.830497,No,5,6
5,GetMOre,Cinema Christmas,12-11-2018,Cinema,Email,SkyKids_Active,Sky_Box_Sets_Active,SGE_Active,Yes,Yes,Yes,17261,426,0.025723,0.025822,-0.000099,0.989840,No,6,1
6,GetMOre,Cinema Christmas,12-11-2018,Cinema,Email,SkyKids_Active,Sky_Box_Sets_Active,SGE_Active,Yes,No,No,34891,913,0.017196,0.019715,-0.002519,0.564030,No,7,5
7,GetMOre,Cinema Christmas,12-11-2018,Cinema,Email,SkyKids_Active,Sky_Box_Sets_Active,SGE_Active,Yes,Yes,No,13845,351,0.020007,0.028490,-0.008483,0.264786,No,8,3


In [234]:
logger = create_logger(None)
format_and_push_table(PROJECT_ID_1,DATASET_ID_1,'lightweight_test',campaign_rules,
                      rules_generator.best_features,
                     'Cinema Christmas',
                     'Cinema',
                     '12-11-2018',
                     'GetMOre',
                     'Email',
                     OPTIMUS_COLS,logger,
                     missing_control=True,
                     format_table=True)

23-Oct-19 15:15:41 - Formatting rules table


,L1_programme,L2_Campaign,CAMPAIGN_REPORTING_START_DATE,product,CHANNEL,best_feature_1,best_feature_2,best_feature_3,best_feature_1_rule,best_feature_2_rule,best_feature_3_rule,treatment_sample_size,control_sample_size,treatment_conversion,control_conversion,uplift,p_value,significative,uplift_ranking,treatment_conversion_ranking
5,GetMOre,Cinema Christmas,12-11-2018,Cinema,Email,SkyKids_Active,Sky_Box_Sets_Active,SGE_Active,Yes,Yes,Yes,17261,0,0.025723,0,0.025723,NaN,NaN,1,1
2,GetMOre,Cinema Christmas,12-11-2018,Cinema,Email,SkyKids_Active,Sky_Box_Sets_Active,SGE_Active,No,Yes,Yes,25531,0,0.021033,0,0.021033,NaN,NaN,2,2
7,GetMOre,Cinema Christmas,12-11-2018,Cinema,Email,SkyKids_Active,Sky_Box_Sets_Active,SGE_Active,Yes,Yes,No,13845,0,0.020007,0,0.020007,NaN,NaN,3,3
0,GetMOre,Cinema Christmas,12-11-2018,Cinema,Email,SkyKids_Active,Sky_Box_Sets_Active,SGE_Active,Yes,No,Yes,25450,0,0.019804,0,0.019804,NaN,NaN,4,4
6,GetMOre,Cinema Christmas,12-11-2018,Cinema,Email,SkyKids_Active,Sky_Box_Sets_Active,SGE_Active,Yes,No,No,34891,0,0.017196,0,0.017196,NaN,NaN,5,5
4,GetMOre,Cinema Christmas,12-11-2018,Cinema,Email,SkyKids_Active,Sky_Box_Sets_Active,SGE_Active,No,Yes,No,29775,0,0.017162,0,0.017162,NaN,NaN,6,6
1,GetMOre,Cinema Christmas,12-11-2018,Cinema,Email,SkyKids_Active,Sky_Box_Sets_Active,SGE_Active,No,No,Yes,281759,0,0.014236,0,0.014236,NaN,NaN,7,7
3,GetMOre,Cinema Christmas,12-11-2018,Cinema,Email,SkyKids_Active,Sky_Box_Sets_Active,SGE_Active,No,No,No,447143,0,0.009804,0,0.009804,NaN,NaN,8,8


or save it in append mode to GCP

In [235]:
format_and_push_table(PROJECT_ID_1,DATASET_ID_1,'lightweight_test_1',campaign_rules,
                      rules_generator.best_features,
                     'Cinema Christmas',
                     'Cinema',
                     '12-11-2018',
                     'GetMOre',
                     'Email',
                     OPTIMUS_COLS,logger,
                     missing_control=True,
                     format_table=False)

23-Oct-19 15:16:05 - Saving table to GCP


## Orchestration

We finally build an orchestration method to run the full pipeline

In [33]:
def last_day_of_month(any_day):
    next_month = any_day.replace(day=28) + datetime.timedelta(days=4)  # this will never fail
    return next_month - datetime.timedelta(days=next_month.day)

In [34]:
def create_prop_mart_date(subset_row):

    prop_mart_date = pd.to_datetime('{}-{}-{}'.format(subset_row['start_date'][0].year,
                                                      subset_row['start_date'][0].month,
                                                      subset_row['start_date'][0].day))

    prop_mart_date = prop_mart_date+dateutil.relativedelta.relativedelta(months=-1)
    prop_mart_date=last_day_of_month(prop_mart_date).strftime('%Y-%m-%d')
    return prop_mart_date

In [35]:
def provide_target_flag(subset_row):
    
    if subset_row['campaign_program']=='skyOptimusDMO':
        if subset_row['product'] in ['TV Pending Cancel', 'TV Active Block  ']:
            return ['TV_REACTIVATE']
        elif subset_row['product'] in ['TV Winback       ']:
            return ['TV_REINSTATE']
        elif subset_row['product'] in ['BB Pending Cancel', 'BB Active Block ']:
            return ['BB_REACTIVATE']
        elif subset_row['product'] in ['BB Winback       ']:
            return 'BB_REINSTATE'
        elif subset_row['campaign_name'] == 'Product Downgrades':
            return ['SPORTS_REINSTATE','CINEMA_REINSTATE', 'TOPTIER_REINSTATE','HD_REINSTATE','SGE_REINSTATE'
                    'BOXSETS_REINSTATE', 'UOD_REINSTATE' ,'QMS_REINSTATE', 'KIDS_REINSTATE','SKYPLUSMS_REINSTATE']
    else:
        return ['sale_target']

In [36]:
def update_list_of_campaigns(project_id, bucket_id: str, campaigns_bkup_table_name: str,
                             list_of_analyzed_campaigns, logger=None):
    
    if logger is not None:
        logger.info('Updating the list of analyzed campaigns')
    fs=gcsfs.GCSFileSystem(project=project_id)
    table_path = os.path.join(bucket_id,campaigns_bkup_table_name)

    if fs.exists('gs://'+table_path):
        backup_table = pd.read_csv('gs://'+table_path)
        if len(list_of_analyzed_campaigns.columns.difference(backup_table.columns)):
            raise ValueError('The campaigns list table and backup table must have the same type of columns')
        
        updated_table = pd.concat([backup_table,list_of_analyzed_campaigns])
        updated_table.reset_index(inplace=True,drop=True)
        updated_table.to_csv('gs://'+table_path,index=False)
    else:

        list_of_analyzed_campaigns.to_csv('gs://'+table_path,index=False)
    

In [43]:
BUCKET_ID

'fci02/test_and_learn'

In [55]:
update_list_of_campaigns(PROJECT_ID_1,BUCKET_ID,'babo.csv',campaigns_list)

In [56]:
pd.read_csv('gs://'+BUCKET_ID+'/babo.csv')

,campaign_program,campaign_name,start_date,journey,product,channel
0,skyOptimusDMO,Winback,2018-10-15,CAM0002279 DMO Winback DM OptIn UK 2018,TV Winback,DM Only
1,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback EM OptIn UK 2018,TV Winback,EM Only
2,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM OptIn UK 2018,TV Winback,DM Only
3,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK March Re...,TV Winback,EM&DM
4,skyOptimusDMO,Winback,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,TV Winback,EM&DM
...,...,...,...,...,...,...
331,skyOptimusGetMore,Sports Cricket Upsell,2019-08-06,CAM0002821 Week 33 Sports Cricket Upsell,Sports,UNKNOWN
332,skyOptimusGetMore,Mobile Switch and Watch,2019-08-08,CAM0002944_Acquisition_Sky Mobile_UK_1920_Q3_O...,Mobile,EM&DM
333,skyOptimusGetMore,Sky Stores Dumbo Hobbs and Shaw,2019-08-08,CAM0002829_Dumbo & Hobbs and Shaw_EMoptin,Broadband,EM Only
334,skyOptimusGetMore,Mobile Switch and Watch,2019-08-08,CAM0002944_Acquisition_Sky Mobile_UK_1920_Q3_O...,Mobile,DM Only


In [101]:
?pull_table

Signature:
pull_table(
    campaign_data_paths,
    prop_mart_data_paths,
    subset_row,
    target_labels,
    optimus_columns,
    prop_mart_features,
    base_date_label,
    prop_mart_date,
    relation,
    control_group_name,
    analysis_level,
    table_name,
    bucket_id,
    step=None,
)
Docstring: <no docstring>
File:      ~/decisioning-analysis/test_and_learn/development/<ipython-input-97-fd29c5f6dcb4>
Type:      function


In [97]:
def pipeline_orchestration(campaign_data_paths, prop_mart_data_paths,
                           bucket_id: str, campaigns_bkup_table_name: str,
                           optimus_columns, prop_mart_features,
                           base_date_label,
                           control_group_name, treatment_group_name,
                           mapping_dict,
                           tree_pms,
                           out_table_name,
                           default_ctr_table_name='Campaign_ctr_tmp', default_treat_table_name='Campaign_treat_tmp',
                           performance_folder=None, log_file=None,
                           tuning=False,
                           test_size=.3,
                           rstate=42,
                           alpha=.05,
                           show=False,
                           check_point_value=5,
                           granularity_levels=['overall', 'channel','journey','step','variant']):
    
    ti = time.time()
    
    # define logger
    logger = create_logger(log_file_name=log_file)
    
    try:
        # inspect all campaigns
        logger.info('Creating list of campaigns table')
        campaigns_list = get_campaigns_relevant_info(campaign_data_paths['project_id'], 
                                                     campaign_data_paths['dataset_id'],
                                                     campaign_data_paths['table_id'],
                                                     optimus_columns['program_label'],
                                                     optimus_columns['campaign_label'],
                                                     optimus_columns['start_date_label'],
                                                     optimus_columns['product_label'],
                                                     optimus_columns['journey_label'],
                                                     optimus_columns['step_label'],
                                                     optimus_columns['variant_label'])
        print(campaigns_list.shape)
        # check with existing backup table: if it does not exist, then we will create it.
        logger.info('Skimming already analyzed tables')
        skimmed_campaign_list = get_skim_campaigns(campaigns_list,
                                                   campaign_data_paths['project_id'],
                                                   bucket_id,
                                                   campaigns_bkup_table_name)
        print(skimmed_campaign_list.shape)
        if not len(skimmed_campaign_list):
            logger.warning('No new campaigns detected. Stopping evaluation.')
            return 0

        to_analyze = skimmed_campaign_list[['campaign_program','campaign_name','start_date','product']].drop_duplicates(subset=['campaign_program','campaign_name','start_date','product'])  

        # loop over campaigns that have not been analyzed
        logger.info('Analyzing {} campaigns'.format(len(to_analyze)))
        list_of_tables = []
        list_of_performances = []
        list_of_analyzed_campaigns = []

        for idx_value, idx in enumerate(to_analyze.index,1):
            t0 = time.time()

            logger.info('Analyzing campaign\n\n{}\n\n'.format(to_analyze.loc[idx,:]))

            for granularity_level in granularity_levels:

                logger.info('##################### Performing analysis at {} level #####################'.format(granularity_level))
                granularity_level_table,analysis_level = create_right_granularity_level(skimmed_campaign_list,to_analyze.loc[[idx],['campaign_program','campaign_name']], granularity_level)
                list_of_analyzed_campaigns.append(create_right_granularity_level(skimmed_campaign_list,to_analyze.loc[[idx],['campaign_program','campaign_name']], 'variant')[0])
                print(analysis_level)
                # construct the right date for pro-mart data
                prop_mart_date = create_prop_mart_date(granularity_level_table)
                logger.info('Using prop-mart snapshot at {}'.format(prop_mart_date))

                for jdx in granularity_level_table.index:

                    subset_row = granularity_level_table.loc[jdx,:]
                    target_var_list = provide_target_flag(subset_row)
                    logger.info('Relevant target(s) for this campaign: {}'.format(target_var_list))

                    if granularity_level in ['overall', 'channel','journey']:
                        logger.info('Selecting journey(s) {}'.format(subset_row['journey']))
                    else:
                        logger.info('Selecting {} {}'.format(granularity_level,subset_row[granularity_level]))

                    # Control
                    logger.info('Building control table')
                    if analysis_level=='variant_label':
                        step=subset_row['step'][0]
                    else:
                        step=None

                    ctr_table = pull_table(campaign_data_paths, prop_mart_data_paths, subset_row,target_var_list,analysis_level,
                                           optimus_columns, prop_mart_features,
                                           base_date_label, prop_mart_date,
                                           '!=', control_group_name[subset_row['campaign_program']], default_ctr_table_name,
                                           bucket_id,step)

                    #Treatment
                    logger.info('Building treatment table')
                    treat_table = pull_table(campaign_data_paths, prop_mart_data_paths, subset_row,target_var_list,analysis_level,
                                             optimus_columns, prop_mart_features,
                                             base_date_label, prop_mart_date,
                                             '=', control_group_name[subset_row['campaign_program']], default_treat_table_name,
                                             bucket_id)
                    logger.info('Control: {} sample points --- Treatment: {} sample points'.format(len(ctr_table),len(treat_table)))
                    logger.info('Control: {}% class imbalance --- Treatment: {}% class imbalance'.format(round(100*ctr_table.target.mean(),3),round(100*treat_table.target.mean(),3)))
                    missing_control=False
                    if not len(treat_table): 
                        logger.warning('Missing treatment group for this campaign. Skipping to next campaign')
                        continue
                    elif (not len(ctr_table)):
                        logger.warning('Missing control group for this campaign. Analysis will focus just on conversion rate')
                        ctr_table = treat_table.copy()
                        missing_control = True

                    # Preprocessing
                    logger.info('Data preprocessing')
                    preprocess_data(ctr_table,mapping_dict,[base_date_label, optimus_columns['account_number_label'],
                                                        optimus_columns['product_label'],optimus_columns['campaign_label'],'target'])
                    post_process_feat = preprocess_data(treat_table,mapping_dict,[base_date_label, optimus_columns['account_number_label'],
                                                        optimus_columns['product_label'],optimus_columns['campaign_label'],'target'])
                    post_process_feat = [feature for feature in post_process_feat if feature in list(ctr_table.columns)]

                    logger.info('Recomputing tables size after preprocessing')
                    logger.info('Control: {} sample points --- Treatment: {} sample points'.format(len(ctr_table),len(treat_table)))
                    logger.info('Control: {}% class imbalance --- Treatment: {}% class imbalance'.format(round(100*ctr_table.target.mean(),3),round(100*treat_table.target.mean(),3)))           
                    if not treat_table.target.mean():
                        logger.error('No conversion detected in the treatment set. Skipping to next campaign')
                        continue
                    elif len(treat_table)<1000:
                        logger.error('Treatment table too small (less than 1000). Skipping to next campaign') 
                        continue
                    elif not(len(ctr_table)):
                        logger.warning('Missing control group for this campaign. Analysis will focus just on conversion rate')
                        ctr_table = treat_table.copy()
                        missing_control = True

                    # Tree analysis
                    logger.info('Running tree analysis')
                    tree_pms.update({'target_feature':'target',
                                    'input_features':post_process_feat,
                                    'logger':logger})
                    rules_generator = TreeRulesExtractor(**tree_pms)

                    rules_generator.forest_tuning(treat_table,test_size,rstate,tuning=tuning,display_frame=show)
                    campaign_rules = rules_generator.tree_rules_generation(treat_table,ctr_table,
                                                                           significance=alpha,print_tree=show)
                    # rules
                    formatted_table = format_and_push_table(campaign_data_paths['project_id'], campaign_data_paths['dataset_id'],out_table_name,campaign_rules,
                                                            rules_generator.best_features,
                                                            subset_row,granularity_level,
                                                            optimus_columns,logger,
                                                            missing_control=missing_control,
                                                            format_table=True)
                    list_of_tables.append(formatted_table)

                    #performances
                    performance_frame = pd.DataFrame(rules_generator.forest_summary)
                    performance_frame = format_performance_frame(performance_frame,subset_row,granularity_level)
                    list_of_performances.append(performance_frame)

            delta = time.time()-t0
            logger.info('Single campaign analysis completed: elapsed time = {} seconds'.format(round(delta,3)))

            if not(idx_value%check_point_value):
                # collect campaigns analysis performance within a unique dataframe
                # save to GCP
                batch_save(idx_value, bucket_id,
                           list_of_performances, list_of_tables,
                           logger,
                           campaign_data_paths,  out_table_name)
                # update list of analyzed campaigns
                list_of_analyzed_campaigns=pd.concat(list_of_analyzed_campaigns)
                update_list_of_campaigns(campaign_data_paths['project_id'], bucket_id, campaigns_bkup_table_name,
                                         list_of_analyzed_campaigns, logger)

                list_of_performances = []
                list_of_tables = []
                list_of_analyzed_campaigns = []

        # collect campaigns analysis performance within a unique dataframe
        # save to GCP
        batch_save(idx_value, bucket_id,
                   list_of_performances, list_of_tables,
                   logger,
                   campaign_data_paths,  out_table_name)
        # update list of analyzed campaigns
        list_of_analyzed_campaigns=pd.concat(list_of_analyzed_campaigns)
        update_list_of_campaigns(campaign_data_paths['project_id'], bucket_id, campaigns_bkup_table_name,
                                 list_of_analyzed_campaigns, logger)

        delta = time.time()-ti
        logger.info('Full campaign analysis completed: elapsed time = {} seconds'.format(round(delta,3)))
    except Exception as e:
        logger.error('Error',  exc_info=e)

## Single campaign example

In [139]:
CAMPAIGN_DATA_PATHS = {'project_id':'skyuk-uk-decis-models-01-dev',
                       'dataset_id':'test_and_learn',
                       'table_id':'optimus_base_clean'}
PROP_MART_DATA_PATHS = {'project_id':'skyuk-uk-decis-models-01-dev',
                       'dataset_id':'uk_inp_prodrecs_ic',
                       'table_id':'Propensity_Model_Mart_Training'} 

OPTIMUS_COLS = {'program_label':'L1_programme',
                'campaign_label':'L2_Campaign',
                'journey_label':'L3_Journey',
                'step_label':'L4_Step',
                'variant_label':'L5_variant',
                'product_label':'product',
                'account_number_label':'account_number',
                'sale_vol_label':'sale_vol',
                'sale_aux_vol_label':'sale_auxillary_vol',
                'cell_label':'cell_type',
                'control_label':'control_type2',
                'start_date_label':'CAMPAIGN_REPORTING_START_DATE',
                'channel_label':'CHANNEL'}

PROPMART_FEAT = ['Base_Dt',
                 'account_number',
                 
                 'TAs_in_last_14d',
                 'BB_Curr_Contract_Intended_End_Dt',
                 'DTV_Curr_Contract_Intnd_End_Dt',
                 'Curr_Offer_Intended_end_Dt_DTV',
                 'Curr_Offer_Intended_end_Dt_BB',
                 'BB_Last_Activation_Dt',
                 'DTV_Last_Activation_Dt',
                 
                 'DTV_Active',
                 'BB_Active',
                 'Sports_Active',
                 'Movies_Active',
                 'MS_Active',
                 'SGE_Active',
                 'HD_Active',
                 'Sky_Box_Sets_Active',
                 'UOD_Active',
                 'SkyKids_Active',
                 'Handset_Active',
                 'skyfibre_enabled',
                 
                 'Age',
                 'h_equivalised_income_value',
                 'h_residence_type',
                 'h_number_of_adults',
                 'h_number_of_children_in_hh',
                 'h_presence_of_child_aged_0_4',
                 'h_presence_of_child_aged_5_11',
                 'h_presence_of_child_aged_12_17',
                 
                 'Broadband_Average_Demand',
                 'BT_Consumer_Market_Share',
                 'Sky_Consumer_Market_Share',
                 'Virgin_Consumer_Market_Share',
                 'TalkTalk_Consumer_Market_Share',
                 
                 
                 'Cricket_Sports_Pref',
                 'Football_Sports_Pref',
                 'Rugby_Union_Sports_Pref',
                 'Rugby_League_Sports_Pref',
                 'F1_Sports_Pref',
                 'Darts_Sports_Pref',
                 'Tennis_Sports_Pref',
                 'Racing_Sports_Pref',
                 'Boxing_Sports_Pref',
                 'Golf_Sports_Pref',
                 'NFL_Sports_Pref',
                 'Wrestling_Sports_Pref',
                 'ESports_Sports_Pref',
                 'GAA_Sports_Pref',
                 'Motorsport_Sports_Pref',
                 'Cycling_Sports_Pref',
                 'Netball_Sports_Pref',

                 'Up_Handset',
                 'Up_Mobile ']

BUCKET_ID = 'fci02/test_and_learn'

mapping_dict = {'h_presence_of_child_aged_0_4':{'No':0,'Yes':1},
                'h_presence_of_child_aged_5_11':{'No':0,'Yes':1},
                'h_presence_of_child_aged_12_17':{'No':0,'Yes':1},}

tree_pms = {'n_estimators':[100, 150, 200, 250, 300],
            'min_samples_leaf':[2500, 5000, 10000],
            'max_depth':[2, 3, 4, 5, 10, 20],
            'max_allowed_depth':5,
            'min_allowed_node_size':5000,
            'n_best_features':3,
            'njobs':14}

control_name_dict = {'skyOptimusGetMore':'Campaign control','skyOptimusDMO':'Fallow Group'}

In [140]:
#campaigns_list.head()

In [548]:
campaigns_list.to_csv('gs://'+BUCKET_ID+'/bckup_analyzed_campaigns_full.csv',index=False)

In [144]:
drop=campaigns_list[campaigns_list.campaign_program=='skyOptimusGetMore'].index
subset = campaigns_list.drop(drop)
subset.to_csv('gs://'+BUCKET_ID+'/bckup_analyzed_campaigns_dry_run.csv',index=False)

In [148]:
drop_perf=campaigns_list[np.logical_or(np.logical_or(campaigns_list['campaign_name']=='Winback DTV UK',
                                           campaigns_list['campaign_name']=='Mobile'),
                            campaigns_list['campaign_name']=='Cinema Christmas')].index
perf_subset = campaigns_list.drop(drop_perf)
perf_subset.to_csv('gs://'+BUCKET_ID+'/bckup_analyzed_campaigns_perf_run.csv',index=False)

In [ ]:
pipeline_orchestration(CAMPAIGN_DATA_PATHS,
                       PROP_MART_DATA_PATHS,
                       BUCKET_ID,
                       'bckup_analyzed_campaigns_perf_run.csv',
                       OPTIMUS_COLS,
                       PROPMART_FEAT,
                       'Base_Dt',
                       control_name_dict,
                       'Treatment',
                       mapping_dict,
                       tree_pms,
                       'campaign_forest_performance',
                       tuning=True,
                       show=False,
                       log_file='performance_log.txt',
                       check_point_value=1,granularity_levels=['overall'])

(22046, 8)
(43, 8)
journey_label
Created BigQuery table: skyuk-uk-decis-models-01-dev.test_and_learn.Campaign_ctr_tmp
Exported skyuk-uk-decis-models-01-dev:test_and_learn.Campaign_ctr_tmp to gs://fci02/test_and_learn/Campaign_ctr_tmp.csv
Created BigQuery table: skyuk-uk-decis-models-01-dev.test_and_learn.Campaign_treat_tmp
Exported skyuk-uk-decis-models-01-dev:test_and_learn.Campaign_treat_tmp to gs://fci02/test_and_learn/Campaign_treat_tmp.csv
#################### DATA PREPROCESSING ####################
1) Removing features with high percentage of nulls
2) Features engineering
2a) Mappings
2b) Rounding income value
2c) Creating contract/offer status features
2d) Creating number of days of tenure
     Number of rows lost: 28
     % of rows lost: 8.89%
Data preprocessed in 0.082 seconds
#################### DATA PREPROCESSING ####################
1) Removing features with high percentage of nulls
2) Features engineering
2a) Mappings
2b) Rounding income value
2c) Creating contract/offer 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  min_samples_leaf=2500, max_depth=2, n_estimators=100, total=  35.1s
[CV] min_samples_leaf=2500, max_depth=2, n_estimators=150 ............


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   37.4s remaining:    0.0s


[CV]  min_samples_leaf=2500, max_depth=2, n_estimators=150, total=  41.1s
[CV] min_samples_leaf=2500, max_depth=2, n_estimators=200 ............
[CV]  min_samples_leaf=2500, max_depth=2, n_estimators=200, total=  46.5s
[CV] min_samples_leaf=2500, max_depth=2, n_estimators=250 ............
[CV]  min_samples_leaf=2500, max_depth=2, n_estimators=250, total=  53.9s
[CV] min_samples_leaf=2500, max_depth=2, n_estimators=300 ............
[CV]  min_samples_leaf=2500, max_depth=2, n_estimators=300, total= 1.0min
[CV] min_samples_leaf=5000, max_depth=2, n_estimators=100 ............
[CV]  min_samples_leaf=5000, max_depth=2, n_estimators=100, total=  34.5s
[CV] min_samples_leaf=5000, max_depth=2, n_estimators=150 ............
[CV]  min_samples_leaf=5000, max_depth=2, n_estimators=150, total=  41.0s
[CV] min_samples_leaf=5000, max_depth=2, n_estimators=200 ............
[CV]  min_samples_leaf=5000, max_depth=2, n_estimators=200, total=  49.2s
[CV] min_samples_leaf=5000, max_depth=2, n_estimators=25

...

['SGE_Active', 'Age', 'h_presence_of_child_aged_5_11']
journey_label
Created BigQuery table: skyuk-uk-decis-models-01-dev.test_and_learn.Campaign_ctr_tmp
Exported skyuk-uk-decis-models-01-dev:test_and_learn.Campaign_ctr_tmp to gs://fci02/test_and_learn/Campaign_ctr_tmp.csv
Created BigQuery table: skyuk-uk-decis-models-01-dev.test_and_learn.Campaign_treat_tmp
Exported skyuk-uk-decis-models-01-dev:test_and_learn.Campaign_treat_tmp to gs://fci02/test_and_learn/Campaign_treat_tmp.csv
#################### DATA PREPROCESSING ####################
1) Removing features with high percentage of nulls
2) Features engineering
2a) Mappings
2b) Rounding income value
2c) Creating contract/offer status features
2d) Creating number of days of tenure
     Number of rows lost: 7676
     % of rows lost: 9.31%
Data preprocessed in 5.274 seconds
#################### DATA PREPROCESSING ####################
1) Removing features with high percentage of nulls
2) Features engineering
2a) Mappings
2b) Rounding inc

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  min_samples_leaf=2500, max_depth=2, n_estimators=100, total=  12.6s
[CV] min_samples_leaf=2500, max_depth=2, n_estimators=150 ............


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   13.3s remaining:    0.0s


[CV]  min_samples_leaf=2500, max_depth=2, n_estimators=150, total=  15.1s
[CV] min_samples_leaf=2500, max_depth=2, n_estimators=200 ............
[CV]  min_samples_leaf=2500, max_depth=2, n_estimators=200, total=  18.8s
[CV] min_samples_leaf=2500, max_depth=2, n_estimators=250 ............
[CV]  min_samples_leaf=2500, max_depth=2, n_estimators=250, total=  20.4s
[CV] min_samples_leaf=2500, max_depth=2, n_estimators=300 ............
[CV]  min_samples_leaf=2500, max_depth=2, n_estimators=300, total=  22.5s
[CV] min_samples_leaf=5000, max_depth=2, n_estimators=100 ............
[CV]  min_samples_leaf=5000, max_depth=2, n_estimators=100, total=  12.9s
[CV] min_samples_leaf=5000, max_depth=2, n_estimators=150 ............
[CV]  min_samples_leaf=5000, max_depth=2, n_estimators=150, total=  15.1s
[CV] min_samples_leaf=5000, max_depth=2, n_estimators=200 ............
[CV]  min_samples_leaf=5000, max_depth=2, n_estimators=200, total=  17.2s
[CV] min_samples_leaf=5000, max_depth=2, n_estimators=25

In [147]:
campaigns_list

,campaign_program,campaign_name,start_date,journey,step,variant,product,channel
0,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,OPTIMUS_WINBACK_JANUARY_EM1_3127-4_(DM241975)_...,TV Winback,EM&DM
1,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,OPTIMUS_WINBACK_JANUARY_EM8_3127-8_(DM243684)_...,TV Winback,EM&DM
2,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,OPTIMUS_WINBACK_JANUARY_EM8_3127-8_(DM243684)_...,TV Winback,EM&DM
3,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,OPTIMUS_WINBACK_JANUARY_EM8_3127-8_(DM243684)_...,TV Winback,EM&DM
4,skyOptimusDMO,Winback DTV UK,2018-10-15,CAM0002279 DMO Winback DM EM OptIn UK 2018,CAM0002279_EM_Ents_Reactive_DMEMOptin_Marketin...,OPTIMUS_WINBACK_JANUARY_EM8_3127-8_(DM243684)_...,TV Winback,EM&DM
...,...,...,...,...,...,...,...,...
22041,skyOptimusGetMore,Mobile Switch and Watch,2019-08-08,CAM0002944_Acquisition_Sky Mobile_UK_1920_Q3_O...,DM_CAM0002944_Sky_Mobile_Switch_Watch_July_201...,CAM0002944_SKY_MOBILE_SWITCH_AND_SAVE_DM_2019_...,Mobile,DM Only
22042,skyOptimusGetMore,Sky Stores Dumbo Hobbs and Shaw,2019-08-08,CAM0002829_Dumbo & Hobbs and Shaw_EMoptin,CAM0002829_Dumbo & Hobbs and Shaw_EMoptin-3,CAM0002829_SS_DHS_EM1A_B_CAM0002829_Short-Term...,Broadband,EM Only
22043,skyOptimusGetMore,Mobile Switch and Watch,2019-08-08,CAM0002944_Acquisition_Sky Mobile_UK_1920_Q3_O...,DM_CAM0002944_Sky_Mobile_Switch_Watch_July_201...,CAM0002944_SKY_MOBILE_SWITCH_AND_SAVE_DM_2019_...,Mobile,DM Only
22044,skyOptimusGetMore,Sports Range ELP,2019-08-26,CAM0002894 Week 36 Sports Range ELP,CAM0002894 Week 36 Sports Range ELP DM OPT IN DM,CAM0002894_WK36_RANGE_ELP_N/A20190816,Sports,UNKNOWN


In [149]:
pd.read_csv('gs://fci02/test_and_learn/campaign_analysis_perf_2019-11-5_check_point_3.csv')

,Unnamed: 0,minimal_forest,minimal_tree,L2_Campaign,product,CAMPAIGN_REPORTING_START_DATE,L1_programme,CHANNEL,L3_Journey,L4_Step,L5_variant
0,max_depth,5.000000,5.000000,Winback DTV UK,TV Winback,2018-10-15,skyOptimusDMO,all,all,all,all
1,min_samples_leaf,5000.000000,5000.000000,Winback DTV UK,TV Winback,2018-10-15,skyOptimusDMO,all,all,all,all
2,n_estimators,100.000000,1.000000,Winback DTV UK,TV Winback,2018-10-15,skyOptimusDMO,all,all,all,all
3,test,0.589245,0.564912,Winback DTV UK,TV Winback,2018-10-15,skyOptimusDMO,all,all,all,all
4,train,0.591608,0.569671,Winback DTV UK,TV Winback,2018-10-15,skyOptimusDMO,all,all,all,all
5,max_depth,5.000000,5.000000,Mobile,Mobile,2018-11-30,skyOptimusGetMore,all,all,all,all
6,min_samples_leaf,5000.000000,5000.000000,Mobile,Mobile,2018-11-30,skyOptimusGetMore,all,all,all,all
7,n_estimators,100.000000,1.000000,Mobile,Mobile,2018-11-30,skyOptimusGetMore,all,all,all,all
8,test,0.642859,0.615502,Mobile,Mobile,2018-11-30,skyOptimusGetMore,all,all,all,all
9,train,0.654933,0.621205,Mobile,Mobile,2018-11-30,skyOptimusGetMore,all,all,all,all


In [159]:
pd.unique(campaigns_list['product'])

array(['TV Winback       ', 'TV Pending Cancel', 'TV Active Block  ',
       'BB Pending Cancel', 'Mobile           ', 'Cinema           ',
       'UOD              ', 'Top Tier         ', 'Boxsets          ',
       'HD               ', 'Kids             ', 'Sky+ Multiscreen ',
       'Q Experience     ', 'Sky Go Extra     ', 'Sports           ',
       'Broadband        ', 'Chelsea TV       ', 'MUTV             ',
       'Sky Go           ', 'QMS              ', 'Fibre            '],
      dtype=object)

In [173]:
import dateutil.relativedelta
str(campaigns_list['start_date'].values[0] - dateutil.relativedelta.relativedelta(months=6))

'2018-04-15'

In [171]:
str(campaigns_list['start_date'].values[0])

'2018-10-15'

In [235]:
campaigns_list['max_step_end_reporting_date']-datetime.date.today()

0       -306 days
1       -285 days
2       -306 days
3       -285 days
4       -306 days
           ...   
22041    -82 days
22042    -97 days
22043    -82 days
22044    -72 days
22045    -72 days
Name: max_step_end_reporting_date, Length: 22046, dtype: timedelta64[ns]

In [237]:
pd.to_datetime(campaigns_list['max_step_end_reporting_date']).dt.date-datetime.date.today()

0       -306 days
1       -285 days
2       -306 days
3       -285 days
4       -306 days
           ...   
22041    -82 days
22042    -97 days
22043    -82 days
22044    -72 days
22045    -72 days
Name: max_step_end_reporting_date, Length: 22046, dtype: timedelta64[ns]

In [319]:
not 0.0

True